In [1]:
#| default_exp models.oak

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
#| export
import torch, re, inspect, pickle, os, torch.nn as nn, math
import torch.nn.functional as F
from dataclasses import dataclass
from typing import Optional, List, Tuple, Mapping, Any, Union
from transformers import (
    PretrainedConfig,
    DistilBertForMaskedLM,
    DistilBertModel,
    DistilBertPreTrainedModel,
    DistilBertConfig,
)
from transformers.utils.generic import ModelOutput
from transformers.activations import get_activation

from fastcore.meta import *
from fastcore.utils import *

from xcai.losses import *
from xcai.core import store_attr
from xcai.learner import XCDataParallel
from xcai.models.modeling_utils import *

In [4]:
from nbdev.showdoc import *
import nbdev; nbdev.nbdev_export()

In [5]:
import copy
from transformers import AutoConfig
from xcai.block import *
from xcai.main import *

## Setup

In [6]:
from xcai.core import prepare_batch

In [7]:
os.environ['CUDA_VISIBLE_DEVICES'] = "0,1"

In [8]:
data_dir = '/Users/suchith720/Projects/data'
config_file = 'wikiseealsotitles'
config_key = 'data_meta'

mname = 'sentence-transformers/msmarco-distilbert-base-v4'

pkl_dir = '/Users/suchith720/Projects/data/processed/mogicX'
os.makedirs(pkl_dir, exist_ok=True)

pkl_file = f'{pkl_dir}/wikiseealsotitles_data_distilbert-base-uncased_sxc.joblib'

In [9]:
block = build_block(pkl_file, config_file, True, config_key, data_dir=data_dir, n_slbl_samples=1, do_build=False, 
                    main_oversample=True, meta_oversample=True, return_scores=True)

In [10]:
block.train.dset.meta['neg_meta'] = copy.deepcopy(block.train.dset.meta['cat_meta'])
block.train.dset.meta['neg_meta'].prefix = 'neg'

In [11]:
batch = block.train.one_batch(4)

In [12]:
for i,batch in enumerate(block.train.dl):
    if i > 2: break

In [20]:
batch.keys()

dict_keys(['data_idx', 'data_identifier', 'data_input_text', 'data_input_ids', 'data_attention_mask', 'plbl2data_idx', 'plbl2data_data2ptr', 'lbl2data_idx', 'lbl2data_scores', 'lbl2data_data2ptr', 'lbl2data_identifier', 'lbl2data_input_text', 'lbl2data_input_ids', 'lbl2data_attention_mask', 'pcat2data_idx', 'pcat2data_data2ptr', 'cat2data_idx', 'cat2data_scores', 'cat2data_data2ptr', 'cat2data_identifier', 'cat2data_input_text', 'cat2data_input_ids', 'cat2data_attention_mask', 'pcat2lbl_idx', 'pcat2lbl_lbl2ptr', 'cat2lbl_idx', 'cat2lbl_scores', 'cat2lbl_lbl2ptr', 'cat2lbl_identifier', 'cat2lbl_input_text', 'cat2lbl_input_ids', 'cat2lbl_attention_mask', 'cat2lbl_data2ptr', 'pcat2lbl_data2ptr', 'pneg2data_idx', 'pneg2data_data2ptr', 'neg2data_idx', 'neg2data_scores', 'neg2data_data2ptr', 'neg2data_identifier', 'neg2data_input_text', 'neg2data_input_ids', 'neg2data_attention_mask', 'pneg2lbl_idx', 'pneg2lbl_lbl2ptr', 'neg2lbl_idx', 'neg2lbl_scores', 'neg2lbl_lbl2ptr', 'neg2lbl_identifier'

In [14]:
[o for o in batch.keys() if 'scores' in o]

['lbl2data_scores',
 'cat2data_scores',
 'cat2lbl_scores',
 'neg2data_scores',
 'neg2lbl_scores']

## CrossAttention

In [15]:
#| export
class CrossAttention(nn.Module):
    
    def __init__(self, config: PretrainedConfig):
        super().__init__()
        self.config, self.n_h, self.dim = config, config.n_heads, config.dim
        self.dropout = nn.Dropout(p=config.attention_dropout)

        if self.dim % self.n_h != 0:
            raise ValueError(f"self.n_heads: {self.n_h} must divide self.dim: {self.dim} evenly.")
            
        self.q = nn.Linear(in_features=config.dim, out_features=config.dim)
        self.k = nn.Linear(in_features=config.dim, out_features=config.dim)
        self.v = nn.Linear(in_features=config.dim, out_features=config.dim)
        self.o = nn.Linear(in_features=config.dim, out_features=config.dim)

    @torch.no_grad()
    def post_init(self):
        nn.init.eye_(self.q.weight.data)
        nn.init.eye_(self.k.weight.data)
        nn.init.eye_(self.v.weight.data)
        nn.init.eye_(self.o.weight.data)

    def forward(
        self, 
        q: torch.Tensor,
        q_m: torch.Tensor,
        k: torch.Tensor, 
        k_m: torch.Tensor,
        output_attentions:Optional[bool] = False,
    ):
        bs, q_len, dim = q.size()
        v, k_len = k, k.size(1) 

        h_dim = self.dim//self.n_h

        def shape(x: torch.Tensor): return x.view(bs, -1, self.n_h, h_dim).transpose(1, 2)

        def unshape(x: torch.Tensor): return x.transpose(1, 2).contiguous().view(bs, -1, self.n_h * h_dim)

        q = shape(self.q(q))  # (bs, n_h, q_len, h_dim)
        k = shape(self.k(k))  # (bs, n_h, k_len, h_dim)
        v = shape(self.v(v))  # (bs, n_h, k_len, h_dim)

        q = q / math.sqrt(h_dim)  # (bs, n_h, q_len, h_dim)
        sc = torch.matmul(q, k.transpose(2, 3))  # (bs, n_h, q_len, k_len)
        
        q_m, k_m = q_m.view(bs, 1, -1, 1).to(q.dtype), k_m.view(bs, 1, 1, -1).to(q.dtype)
        mask = torch.matmul(q_m, k_m).expand_as(sc)  # (bs, n_h, q_len, k_len)
        
        sc = sc.masked_fill(mask == 0, torch.tensor(torch.finfo(sc.dtype).min))  # (bs, n_h, q_len, k_len)

        w = nn.functional.softmax(sc, dim=-1)  # (bs, n_h, q_len, k_len)
        w = self.dropout(w)  # (bs, n_h, q_len, k_len)

        o = self.o(unshape(torch.matmul(w, v))) # (bs, q_len, dim)
        
        if output_attentions: return (o, w)
        else: return (o,)
        

### Example

In [ ]:
config = AutoConfig.from_pretrained('distilbert-base-uncased')
fuser = CrossAttention(config)

In [ ]:
bsz, data_seq_len, n_meta, dim, dtype = 2, 3, 2, config.dim, torch.float32
data, meta = torch.randn(bsz, data_seq_len, dim, dtype=dtype), torch.randn(bsz, n_meta, dim, dtype=dtype)
data_mask = torch.randint(0, 2, size=(bsz,data_seq_len), dtype=dtype)
meta_mask = torch.randint(0, 2, size=(bsz,n_meta), dtype=dtype)

In [ ]:
o = fuser(data, data_mask, meta, meta_mask)

In [ ]:
o[0].shape

torch.Size([2, 3, 768])

## NormCrossAttention

In [ ]:
#| export
class NormCrossAttention(nn.Module):
    
    def __init__(self, config: PretrainedConfig, tau:Optional[float]=0.1, dropout:Optional[float]=0.1):
        super().__init__()
        self.tau = nn.Parameter(torch.tensor(tau, dtype=torch.float32))
        
        self.config, self.n_h, self.dim = config, config.n_heads, config.dim
        self.dropout = nn.Dropout(p=dropout)

        if self.dim % self.n_h != 0:
            raise ValueError(f"self.n_heads: {self.n_h} must divide self.dim: {self.dim} evenly.")
            
        self.q = nn.Linear(in_features=config.dim, out_features=config.dim)
        self.k = nn.Linear(in_features=config.dim, out_features=config.dim)
        self.v = nn.Linear(in_features=config.dim, out_features=config.dim)
        self.o = nn.Linear(in_features=config.dim, out_features=config.dim)

    def post_init(self):
        self.q.weight.data = torch.eye(self.q.out_features, self.q.in_features, dtype=self.q.weight.dtype)
        self.k.weight.data = torch.eye(self.k.out_features, self.k.in_features, dtype=self.k.weight.dtype)
        self.v.weight.data = torch.eye(self.v.out_features, self.v.in_features, dtype=self.v.weight.dtype)
        self.o.weight.data = torch.eye(self.o.out_features, self.o.in_features, dtype=self.o.weight.dtype)

    def forward(
        self, 
        q: torch.Tensor,
        q_m: torch.Tensor,
        k: torch.Tensor, 
        k_m: torch.Tensor,
        output_attentions:Optional[bool] = False,
    ):
        bs, q_len, dim = q.size()
        v, k_len = k, k.size(1) 

        h_dim = self.dim//self.n_h

        def shape(x: torch.Tensor): return x.view(bs, -1, self.n_h, h_dim).transpose(1, 2)

        def unshape(x: torch.Tensor): return x.transpose(1, 2).contiguous().view(bs, -1, self.n_h * h_dim)

        q = shape(self.q(q))  # (bs, n_h, q_len, h_dim)
        k = shape(self.k(k))  # (bs, n_h, k_len, h_dim)
        v = shape(self.v(v))  # (bs, n_h, k_len, h_dim)

        q = q / math.sqrt(h_dim)  # (bs, n_h, q_len, h_dim)
        sc = torch.matmul(q, k.transpose(2, 3))  # (bs, n_h, q_len, k_len)
        sc = sc * self.tau
        
        q_m, k_m = q_m.view(bs, 1, -1, 1).to(q.dtype), k_m.view(bs, 1, 1, -1).to(q.dtype)
        mask = torch.matmul(q_m, k_m).expand_as(sc)  # (bs, n_h, q_len, k_len)
        
        sc = sc.masked_fill(mask == 0, torch.tensor(torch.finfo(sc.dtype).min))  # (bs, n_h, q_len, k_len)
        
        w = nn.functional.softmax(sc, dim=-1)  # (bs, n_h, q_len, k_len)
        w = self.dropout(w)  # (bs, n_h, q_len, k_len)

        o = self.o(unshape(torch.matmul(w, v))) # (bs, q_len, dim)
        
        if output_attentions: return (o, w)
        else: return (o,)
        

### Example

In [ ]:
config = AutoConfig.from_pretrained('distilbert-base-uncased')
fuser = NormCrossAttention(config)

In [ ]:
bsz, data_seq_len, n_meta, dim, dtype = 2, 3, 2, config.dim, torch.float32

data, meta = torch.randn(bsz, data_seq_len, dim, dtype=dtype), torch.randn(bsz, n_meta, dim, dtype=dtype)
data_mask = torch.randint(0, 2, size=(bsz,data_seq_len), dtype=dtype)
meta_mask = torch.randint(0, 2, size=(bsz,n_meta), dtype=dtype)

In [ ]:
o = fuser(data, data_mask, meta, meta_mask)

In [ ]:
o[0].shape

torch.Size([2, 3, 768])

## Encoder

In [16]:
#| export
class Encoder(DistilBertPreTrainedModel):
    
    def __init__(
        self, 
        config:PretrainedConfig, 
        num_metadata:int,
        resize_length:Optional[int]=None,
        normalize:Optional[bool]=True,
    ):
        super().__init__(config)
        store_attr('normalize')
        self.distilbert = DistilBertModel(config)
        
        self.dr_head = RepresentationHead(config)
        self.dr_fused_head = RepresentationHead(config)
        
        self.meta_head = RepresentationHead(config)
        
        self.cross_head = CrossAttention(config)
        
        self.meta_embeddings = nn.Embedding(num_metadata, config.dim)

        if resize_length is None: self.ones = None
        else: self.register_buffer('ones', torch.ones(resize_length, dtype=torch.long, device=self.device))
        
        self.post_init()

    def freeze_meta_embeddings(self):
        self.meta_embeddings.requires_grad_(False)

    def unfreeze_meta_embeddings(self):
        self.meta_embeddings.requires_grad_(True)

    @torch.no_grad()
    def set_meta_embeddings(self, embed:torch.Tensor):
        self.meta_embeddings.weight.data.copy_(embed)
        
    def get_position_embeddings(self) -> nn.Embedding:
        return self.distilbert.get_position_embeddings()
    
    def resize_position_embeddings(self, new_num_position_embeddings: int):
        self.distilbert.resize_position_embeddings(new_num_position_embeddings)
    
    def encode(self, input_ids:torch.Tensor, attention_mask:torch.Tensor, **kwargs):
        return self.distilbert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            **kwargs
        )
    
    def dr(self, embed:torch.Tensor, attention_mask:torch.Tensor):
        embed = self.dr_head(embed)
        embed = Pooling.mean_pooling(embed, attention_mask)
        return F.normalize(embed, dim=1) if self.normalize else embed

    def dr_fused(self, embed:torch.Tensor):
        embed = self.dr_fused_head(embed)
        return F.normalize(embed, dim=1) if self.normalize else embed

    def meta(self, embed:torch.Tensor, attention_mask:torch.Tensor, normalize:Optional[bool]=True):
        embed = self.meta_head(embed)
        embed = Pooling.mean_pooling(embed, attention_mask)
        return F.normalize(embed, dim=1) if normalize else embed

    def resize(self, idx:torch.Tensor, num_inputs:torch.Tensor):
        if torch.any(num_inputs == 0): raise ValueError("`num_inputs` should be non-zero positive integer.")
        bsz, total_num_inputs = num_inputs.shape[0], idx.shape[0]
        
        self.ones = self.ones.to(idx.device)
        ones = (
            torch.ones(total_num_inputs, dtype=torch.long, device=idx.device) 
            if self.ones is None or self.ones.shape[0] < total_num_inputs else self.ones[:total_num_inputs]
        )

        max_num_inputs = num_inputs.max()
        if (num_inputs == max_num_inputs).all():
            return idx,ones
        
        xnum_inputs = max_num_inputs-num_inputs+1

        inputs_ptr = num_inputs.cumsum(dim=0)-1
        repeat_inputs = ones.scatter(0, inputs_ptr, xnum_inputs)
        
        resized_idx = idx.repeat_interleave(repeat_inputs, dim=0)
        ignore_mask = ones.scatter(0, inputs_ptr, 0).repeat_interleave(repeat_inputs, dim=0).view(bsz, -1)
        ignore_mask[:, -1] = 1; ignore_mask = ignore_mask.flatten()
        
        return resized_idx,ignore_mask

    
    def fuse_meta_into_embeddings(self, data_repr:torch.Tensor, data_mask:torch.Tensor, meta_kwargs:Dict):
        meta_repr = {}
        
        data_fused_repr, data_mask = data_repr.clone().view(-1, 1, self.config.dim), data_mask.view(-1, 1)
        for m_key, m_args in meta_kwargs.items():
            idx = torch.where(m_args['data2ptr'] > 0)[0]
            meta_repr[m_key] = torch.empty(0, self.config.dim).to(data_repr)
            
            if len(idx):
                m_idx,m_repr_mask = self.resize(m_args['idx'], m_args['data2ptr'][idx])
                m_repr = self.meta_embeddings(m_idx)
                m_repr = F.normalize(m_repr, dim=1) if self.normalize else m_repr
                
                m_repr, m_repr_mask = m_repr.view(len(idx), -1, self.config.dim), m_repr_mask.bool().view(len(idx), -1)
                meta_repr[m_key] = m_repr[m_repr_mask]
                
                fused_repr = self.cross_head(data_fused_repr[idx], data_mask[idx], m_repr, m_repr_mask)[0]
                data_fused_repr[idx] += fused_repr
                
        return data_fused_repr.squeeze(), meta_repr

    def forward(
        self, 
        data_input_ids: torch.Tensor, 
        data_attention_mask: torch.Tensor,
        data_aug_meta_prefix: Optional[str]=None,
        data_type:Optional[str]=None,
        data_unnormalized:Optional[bool]=False,
        **kwargs
    ):  
        data_o = self.encode(data_input_ids, data_attention_mask)
        
        if data_type is not None and data_type == "meta":
            data_repr = self.meta(data_o[0], data_attention_mask, not data_unnormalized)
        else: 
            data_repr = self.dr(data_o[0], data_attention_mask)
        
        data_fused_repr = meta_repr = None
        if data_aug_meta_prefix is not None:
            meta_kwargs = Parameters.from_meta_aug_prefix(data_aug_meta_prefix, **kwargs)
            if len(meta_kwargs):
                data_fused_repr, meta_repr = self.fuse_meta_into_embeddings(data_repr, 
                                                                            torch.any(data_attention_mask, dim=1), 
                                                                            meta_kwargs)
                data_fused_repr = self.dr_fused(data_fused_repr)
                
        return EncoderOutput(
            rep=data_repr,
            fused_rep=data_fused_repr,
            meta_repr=meta_repr,
        )
        

## `OAK000`

In [21]:
#| export
class OAK000(nn.Module):
    
    def __init__(
        self, config,

        data_aug_meta_prefix:Optional[str]=None, 
        lbl2data_aug_meta_prefix:Optional[str]=None, 

        data_pred_meta_prefix:Optional[str]=None,
        lbl2data_pred_meta_prefix:Optional[str]=None,
        
        margin:Optional[float]=0.3,
        num_negatives:Optional[int]=5,
        tau:Optional[float]=0.1,
        apply_softmax:Optional[bool]=True,

        calib_margin:Optional[float]=0.3,
        calib_num_negatives:Optional[int]=10,
        calib_tau:Optional[float]=0.1,
        calib_apply_softmax:Optional[bool]=False,
        calib_loss_weight:Optional[float]=0.1,
        use_calib_loss:Optional[float]=False,
        
        meta_loss_weight:Optional[Union[List,float]]=0.3,
        
        use_fusion_loss:Optional[bool]=False,
        fusion_loss_weight:Optional[float]=0.15,

        use_query_loss:Optional[float]=True,
        
        use_encoder_parallel:Optional[bool]=True,
        
        normalize:Optional[bool]=True,
    ):
        super().__init__(config)
        store_attr('meta_loss_weight,fusion_loss_weight,calib_loss_weight')
        store_attr('data_pred_meta_prefix,lbl2data_pred_meta_prefix')
        store_attr('data_aug_meta_prefix,lbl2data_aug_meta_prefix')
        store_attr('use_fusion_loss,use_query_loss,use_calib_loss,use_encoder_parallel')
        store_attr('normalize')
        
        self.encoder = None
        self.rep_loss_fn = MultiTriplet(margin=margin, n_negatives=num_negatives, tau=tau, 
                                        apply_softmax=apply_softmax, reduce='mean')
        self.cab_loss_fn = Calibration(margin=calib_margin, tau=calib_tau, n_negatives=calib_num_negatives, 
                                       apply_softmax=calib_apply_softmax, reduce='mean')
        
    @torch.no_grad()
    def init_retrieval_head(self):
        assert self.encoder is not None, "`self.encoder` is not initialized."
        self.encoder.dr_head.post_init()
        self.encoder.meta_head.post_init()
        self.encoder.dr_fused_head.post_init()

    @torch.no_grad()
    def init_cross_head(self):
        assert self.encoder is not None, "`self.encoder` is not initialized."
        self.encoder.cross_head.post_init()

    def compute_loss(self, inp_repr, targ_repr, targ_ptr, targ_idx, ptarg_ptr, ptarg_idx):
        return self.rep_loss_fn(inp_repr, targ_repr, targ_ptr, targ_idx, ptarg_ptr, ptarg_idx)

    def calibration_loss(self, einp_repr, inp_repr, targ_repr, targ_ptr, targ_idx, ptarg_ptr, ptarg_idx):
        return self.calib_loss_weight * self.cab_loss_fn(einp_repr, inp_repr, targ_repr, targ_ptr, targ_idx, ptarg_ptr, ptarg_idx)
    
    def compute_meta_loss(self, data_repr, lbl2data_repr, **kwargs):
        if self.use_encoder_parallel: 
            encoder = XCDataParallel(module=self.encoder)
        else: encoder = self.encoder
            
        data_meta_inputs = Parameters.from_meta_pred_prefix(self.data_pred_meta_prefix, **kwargs)
        lbl2data_meta_inputs = Parameters.from_meta_pred_prefix(self.lbl2data_pred_meta_prefix, **kwargs)
        meta_inputs = {**data_meta_inputs, **lbl2data_meta_inputs}

        m_lw = Parameters.get_meta_loss_weights(self.meta_loss_weight, len(meta_inputs)) if len(meta_inputs) else []
        
        loss = 0.0
        for inputs,lw in zip(meta_inputs.values(), m_lw):
            if 'lbl2data2ptr' in inputs:
                idx = torch.where(inputs['lbl2data2ptr'])[0]
                if len(idx) > 0:
                    inputs_o = encoder(data_input_ids=inputs['input_ids'], data_attention_mask=inputs['attention_mask'], 
                                       data_type="meta")
                    m_loss = self.rep_loss_fn(lbl2data_repr[idx], inputs_o.rep, inputs['lbl2data2ptr'][idx],
                                              inputs['idx'], inputs['plbl2data2ptr'][idx], inputs['pidx'])
                    loss += lw * m_loss

            elif 'data2ptr' in inputs:
                idx = torch.where(inputs['data2ptr'])[0]
                if len(idx) > 0:
                    inputs_o = encoder(data_input_ids=inputs['input_ids'], data_attention_mask=inputs['attention_mask'], 
                                       data_type="meta")
                    m_loss = self.rep_loss_fn(data_repr[idx], inputs_o.rep, inputs['data2ptr'][idx], inputs['idx'], 
                                              inputs['pdata2ptr'][idx], inputs['pidx'])
                    loss += lw * m_loss       

            else: raise ValueError('Invalid metadata input arguments.')
        return loss

    def compute_fusion_loss(self, data_repr, meta_repr:Dict, prefix:str, **kwargs):
        meta_inputs = Parameters.from_meta_pred_prefix(prefix, **kwargs)
        
        loss = 0.0
        if meta_repr is not None:
            for key,input_repr in meta_repr.items():
                inputs = meta_inputs[key]
                if 'lbl2data2ptr' in inputs:
                    idx = torch.where(inputs['lbl2data2ptr'])[0]
                    if len(idx) > 0:
                        m_loss = self.rep_loss_fn(data_repr[idx], input_repr, inputs['lbl2data2ptr'][idx],
                                                  inputs['idx'], inputs['plbl2data2ptr'][idx], inputs['pidx'])
                        loss += self.fusion_loss_weight * m_loss
    
                elif 'data2ptr' in inputs:
                    idx = torch.where(inputs['data2ptr'])[0]
                    if len(idx) > 0:
                        m_loss = self.rep_loss_fn(data_repr[idx], input_repr, inputs['data2ptr'][idx], inputs['idx'], 
                                                  inputs['pdata2ptr'][idx], inputs['pidx'])
                        loss += self.fusion_loss_weight * m_loss       
    
                else: raise ValueError('Invalid metadata input arguments.')
        return loss


    def get_meta_representation(
        self,
        data_input_ids:Optional[torch.Tensor]=None,
        data_attention_mask:Optional[torch.Tensor]=None,
        **kwargs
    ):
        if self.use_encoder_parallel: 
            encoder = XCDataParallel(module=self.encoder)
        else: encoder = self.encoder
            
        data_o = encoder(data_input_ids=data_input_ids, data_attention_mask=data_attention_mask, 
                         data_unnormalized=True, data_type="meta")
        return XCModelOutput(
            data_repr=data_o.rep,
            data_fused_repr=data_o.fused_rep,
        )
        
    def forward(
        self,
        data_input_ids:Optional[torch.Tensor]=None,
        data_attention_mask:Optional[torch.Tensor]=None,
        lbl2data_data2ptr:Optional[torch.Tensor]=None,
        lbl2data_idx:Optional[torch.Tensor]=None,
        lbl2data_input_ids:Optional[torch.Tensor]=None,
        lbl2data_attention_mask:Optional[torch.Tensor]=None,
        plbl2data_data2ptr:Optional[torch.Tensor]=None,
        plbl2data_idx:Optional[torch.Tensor]=None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
        **kwargs
    ):  
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict
        
        if self.use_encoder_parallel: 
            encoder = XCDataParallel(module=self.encoder)
        else: encoder = self.encoder
        
        data_meta_kwargs = Parameters.from_feat_meta_aug_prefix('data', self.data_aug_meta_prefix, **kwargs)
        data_o = encoder(data_input_ids=data_input_ids, data_attention_mask=data_attention_mask, 
                         data_aug_meta_prefix=self.data_aug_meta_prefix, **data_meta_kwargs)
        
        
        loss = None; lbl2data_o = EncoderOutput()
        if lbl2data_input_ids is not None:
            lbl2data_meta_kwargs = Parameters.from_feat_meta_aug_prefix('lbl2data', self.lbl2data_aug_meta_prefix, **kwargs)
            lbl2data_o = encoder(data_input_ids=lbl2data_input_ids, data_attention_mask=lbl2data_attention_mask, 
                                 data_aug_meta_prefix=self.lbl2data_aug_meta_prefix, **lbl2data_meta_kwargs)
            
            loss = self.compute_loss(data_o.fused_rep, lbl2data_o.rep,lbl2data_data2ptr,lbl2data_idx,
                                     plbl2data_data2ptr,plbl2data_idx)

            if self.use_query_loss:
                loss += self.compute_loss(data_o.rep, lbl2data_o.rep,lbl2data_data2ptr,lbl2data_idx,
                                          plbl2data_data2ptr,plbl2data_idx)

            if self.use_calib_loss:
                loss += self.calibration_loss(data_o.fused_rep, data_o.rep, lbl2data_o.rep,lbl2data_data2ptr,lbl2data_idx,
                                              plbl2data_data2ptr,plbl2data_idx)
            
            loss += self.compute_meta_loss(data_o.fused_rep, lbl2data_o.rep, **kwargs)
            
            if self.use_fusion_loss:
                loss += self.compute_fusion_loss(data_o.fused_rep, data_o.meta_repr, self.data_aug_meta_prefix, **kwargs)
                loss += self.compute_fusion_loss(lbl2data_o.rep, lbl2data_o.meta_repr, self.lbl2data_aug_meta_prefix, **kwargs)
            
            
        if not return_dict:
            o = (data_o.logits,data_o.rep,data_o.fused_rep,lbl2data_o.logits,lbl2data_o.rep,lbl2data_o.fused_rep)
            return ((loss,) + o) if loss is not None else o
        
        
        return XCModelOutput(
            loss=loss,
            
            data_repr=data_o.rep,
            data_fused_repr=data_o.fused_rep,
            
            lbl2data_repr=lbl2data_o.rep,
            lbl2data_fused_repr=lbl2data_o.fused_rep,
        )
        

## `OAK001`

In [14]:
#| export
class OAK001(OAK000, DistilBertPreTrainedModel):
    use_generation,use_representation = False,True
    _tied_weights_keys = ["encoder.distilbert"]

    @delegates(OAK000.__init__)
    def __init__(
        self, 
        config,
        num_metadata:int,
        resize_length:Optional[int]=None,
        **kwargs
    ):
        super().__init__(config, **kwargs)
        self.encoder = Encoder(config, num_metadata=num_metadata, resize_length=resize_length)
        self.post_init(); self.remap_post_init(); self.init_retrieval_head(); self.init_cross_head()

    def remap_post_init(self):
        self.distilbert = self.encoder.distilbert
        

### Example

In [58]:
model = OAK001.from_pretrained('sentence-transformers/msmarco-distilbert-base-v4', batch_size=100, num_batch_labels=5000, 
                               margin=0.3, num_negatives=5, tau=0.1, apply_softmax=True,
                               
                               data_aug_meta_prefix='lnk2data', lbl2data_aug_meta_prefix=None, 
                               data_pred_meta_prefix=None, lbl2data_pred_meta_prefix=None,
                               
                               num_metadata=block.train.dset.meta['lnk_meta'].n_meta, resize_length=5000,
                               
                               calib_margin=0.3, calib_num_negatives=10, calib_tau=0.1, calib_apply_softmax=True, 
                               calib_loss_weight=0.1, use_calib_loss=False,

                               use_query_loss=True,

                               meta_loss_weight=0.3, 
                               
                               fusion_loss_weight=0.1, use_fusion_loss=True,
                               
                               use_encoder_parallel=False)
model.init_retrieval_head()
model.init_cross_head()

Some weights of OAK001 were not initialized from the model checkpoint at sentence-transformers/msmarco-distilbert-base-v4 and are newly initialized: ['encoder.cross_head.k.bias', 'encoder.cross_head.k.weight', 'encoder.cross_head.o.bias', 'encoder.cross_head.o.weight', 'encoder.cross_head.q.bias', 'encoder.cross_head.q.weight', 'encoder.cross_head.v.bias', 'encoder.cross_head.v.weight', 'encoder.dr_fused_head.layer_norm.bias', 'encoder.dr_fused_head.layer_norm.weight', 'encoder.dr_fused_head.projector.bias', 'encoder.dr_fused_head.projector.weight', 'encoder.dr_fused_head.transform.bias', 'encoder.dr_fused_head.transform.weight', 'encoder.dr_head.layer_norm.bias', 'encoder.dr_head.layer_norm.weight', 'encoder.dr_head.projector.bias', 'encoder.dr_head.projector.weight', 'encoder.dr_head.transform.bias', 'encoder.dr_head.transform.weight', 'encoder.meta_embeddings.weight', 'encoder.meta_head.layer_norm.bias', 'encoder.meta_head.layer_norm.weight', 'encoder.meta_head.projector.bias', 'enc

In [59]:
model.encoder.set_meta_embeddings(torch.zeros(656086, 768))

In [60]:
model = model.to('cuda')

In [61]:
b = prepare_batch(model, batch, m_args=[
    'plnk2data_idx', 'plnk2data_data2ptr', 'lnk2data_idx', 'lnk2data_input_ids', 'lnk2data_attention_mask', 
    'lnk2data_data2ptr',
])

In [62]:
o = model(**b.to(model.device))

/scratch/scai/phd/aiz218323/Projects/xcai/xcai/losses.py:22: UserWarning: Sparse CSR tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at ../aten/src/ATen/SparseCsrTensorImpl.cpp:53.)
  return torch.sparse_csr_tensor(data_ptr, data_idx, scores, device=data_ptr.device)


In [63]:
def func():
    import pdb; pdb.set_trace()
    return model(**b.to(model.device))
    

In [64]:
o = func()

> /tmp/ipykernel_5515/3657616883.py(3)func()
      1 def func():
      2     import pdb; pdb.set_trace()
----> 3     return model(**b.to(model.device))
      4 



ipdb>  b model.forward


Breakpoint 1 at /tmp/ipykernel_5515/3933810934.py:143


ipdb>  b model.encoder.forward


Breakpoint 2 at /tmp/ipykernel_5515/4122862164.py:106


...

ipdb>  m_repr_mask.shape


torch.Size([5, 3])


ipdb>  n


> /tmp/ipykernel_5515/4122862164.py(101)fuse_meta_into_embeddings()
     99                 meta_repr[m_key] = m_repr[m_repr_mask]
    100 
--> 101                 fused_repr = self.cross_head(data_fused_repr[idx], data_mask[idx], m_repr, m_repr_mask)[0]
    102                 data_fused_repr[idx] += fused_repr
    103 



ipdb>  q


In [ ]:
o.loss

tensor(0.0670, device='cuda:0', grad_fn=<AddBackward0>)

## `OAK002`

In [ ]:
#| export
class Encoder002(Encoder):

    def __init__(
        self, 
        config,
        cross_tau:Optional[float]=0.1, 
        cross_dropout:Optional[float]=0.1, 
        **kwargs
    ):
        super().__init__(config, **kwargs)
        self.cross_head = NormCrossAttention(config, tau=cross_tau, dropout=cross_dropout)
        self.post_init()
    

In [ ]:
#| export
class OAK002(OAK000, DistilBertPreTrainedModel):
    use_generation,use_representation = False,True
    _tied_weights_keys = ["encoder.distilbert"]

    @delegates(OAK000.__init__)
    def __init__(
        self, 
        config,
        num_metadata:int,
        resize_length:Optional[int]=None,
        
        cross_tau:Optional[float]=0.1,
        cross_dropout:Optional[float]=0.1,
        
        **kwargs
    ):
        super().__init__(config, **kwargs)
        self.encoder = Encoder002(config, cross_tau=cross_tau, cross_dropout=cross_dropout,
                                  num_metadata=num_metadata, resize_length=resize_length)
        self.post_init(); self.remap_post_init(); self.init_retrieval_head(); self.init_cross_head()

    def remap_post_init(self):
        self.distilbert = self.encoder.distilbert
        

### Example

In [ ]:
model = OAK002.from_pretrained('sentence-transformers/msmarco-distilbert-base-v4', batch_size=100, num_batch_labels=5000, 
                               margin=0.3, num_negatives=5, tau=0.1, apply_softmax=True,
                               
                               data_aug_meta_prefix='lnk2data', lbl2data_aug_meta_prefix=None, 
                               data_pred_meta_prefix=None, lbl2data_pred_meta_prefix=None,
                               
                               num_metadata=block.train.dset.meta['cat_meta'].n_meta, resize_length=5000,
                               
                               calib_margin=0.3, calib_num_negatives=10, calib_tau=0.1, calib_apply_softmax=True, 
                               calib_loss_weight=0.1, use_calib_loss=False,

                               cross_tau=1.0, cross_dropout=0.1,

                               use_query_loss=True,

                               meta_loss_weight=0.3, 
                               
                               fusion_loss_weight=0.1, use_fusion_loss=True,
                               
                               use_encoder_parallel=False)
model.init_retrieval_head()
model.init_cross_head()

Some weights of OAK002 were not initialized from the model checkpoint at sentence-transformers/msmarco-distilbert-base-v4 and are newly initialized: ['encoder.cross_head.k.bias', 'encoder.cross_head.k.weight', 'encoder.cross_head.o.bias', 'encoder.cross_head.o.weight', 'encoder.cross_head.q.bias', 'encoder.cross_head.q.weight', 'encoder.cross_head.tau', 'encoder.cross_head.v.bias', 'encoder.cross_head.v.weight', 'encoder.dr_fused_head.layer_norm.bias', 'encoder.dr_fused_head.layer_norm.weight', 'encoder.dr_fused_head.projector.bias', 'encoder.dr_fused_head.projector.weight', 'encoder.dr_fused_head.transform.bias', 'encoder.dr_fused_head.transform.weight', 'encoder.dr_head.layer_norm.bias', 'encoder.dr_head.layer_norm.weight', 'encoder.dr_head.projector.bias', 'encoder.dr_head.projector.weight', 'encoder.dr_head.transform.bias', 'encoder.dr_head.transform.weight', 'encoder.meta_embeddings.weight', 'encoder.meta_head.layer_norm.bias', 'encoder.meta_head.layer_norm.weight', 'encoder.meta_

In [ ]:
model.encoder.set_meta_embeddings(torch.zeros(656086, 768))

In [ ]:
model = model.to('cuda')

In [ ]:
b = prepare_batch(model, batch, m_args=[
    'plnk2data_idx', 'plnk2data_data2ptr', 'lnk2data_idx', 'lnk2data_input_ids', 'lnk2data_attention_mask', 
    'lnk2data_data2ptr',
])

In [ ]:
o = model(**b.to(model.device))

> /tmp/ipykernel_36176/2795290057.py(33)forward()
     31         output_attentions:Optional[bool] = False,
     32     ):
---> 33         bs, q_len, dim = q.size()
     34         v, k_len = k, k.size(1)
     35 



ipdb>  c


> /tmp/ipykernel_36176/2795290057.py(34)forward()
     32     ):
     33         bs, q_len, dim = q.size()
---> 34         v, k_len = k, k.size(1)
     35 
     36         h_dim = self.dim//self.n_h



ipdb>  c


In [ ]:
o.loss

tensor(0.0638, device='cuda:0', grad_fn=<AddBackward0>)

## `OAK003`

In [22]:
#| export
class Encoder003(Encoder):

    def __init__(
        self, 
        config,
        num_metadata:int,
        **kwargs
    ):
        super().__init__(config, num_metadata=num_metadata, **kwargs)
        self.pretrained_meta_embeddings = nn.Embedding(num_metadata, config.dim)
        self.post_init()

    def freeze_pretrained_meta_embeddings(self):
        self.pretrained_meta_embeddings.requires_grad_(False)

    def unfreeze_pretrained_meta_embeddings(self):
        self.pretrained_meta_embeddings.requires_grad_(True)

    @torch.no_grad()
    def set_pretrained_meta_embeddings(self, embed:torch.Tensor):
        self.pretrained_meta_embeddings.weight.data.copy_(embed)

    @torch.no_grad()
    def init_meta_embeddings(self):
        nn.init.zeros_(self.meta_embeddings.weight.data)

    def fuse_meta_into_embeddings(self, data_repr:torch.Tensor, data_mask:torch.Tensor, meta_kwargs:Dict):
        meta_repr = {}
        
        data_fused_repr, data_mask = data_repr.clone().view(-1, 1, self.config.dim), data_mask.view(-1, 1)
        for m_key, m_args in meta_kwargs.items():
            idx = torch.where(m_args['data2ptr'] > 0)[0]
            meta_repr[m_key] = torch.empty(0, self.config.dim).to(data_repr)
            
            if len(idx):
                m_idx,m_repr_mask = self.resize(m_args['idx'], m_args['data2ptr'][idx])
                m_repr = self.meta_embeddings(m_idx) + self.pretrained_meta_embeddings(m_idx)
                m_repr = F.normalize(m_repr, dim=1) if self.normalize else m_repr
                
                m_repr, m_repr_mask = m_repr.view(len(idx), -1, self.config.dim), m_repr_mask.bool().view(len(idx), -1)
                meta_repr[m_key] = m_repr[m_repr_mask]
                
                fused_repr = self.cross_head(data_fused_repr[idx], data_mask[idx], m_repr, m_repr_mask)[0]
                data_fused_repr[idx] += fused_repr
                
        return data_fused_repr.squeeze(), meta_repr
    

In [23]:
#| export
class OAK003(OAK000, DistilBertPreTrainedModel):
    use_generation,use_representation = False,True
    _tied_weights_keys = ["encoder.distilbert"]

    @delegates(OAK000.__init__)
    def __init__(
        self, 
        config,
        num_metadata:int,
        resize_length:Optional[int]=None,
        **kwargs
    ):
        super().__init__(config, **kwargs)
        self.encoder = Encoder003(config, num_metadata=num_metadata, resize_length=resize_length)
        self.post_init(); self.remap_post_init(); self.init_retrieval_head(); self.init_cross_head()

    @torch.no_grad()
    def init_meta_embeddings(self):
        self.encoder.init_meta_embeddings()

    def remap_post_init(self):
        self.distilbert = self.encoder.distilbert
        

### Example

In [ ]:
model = OAK003.from_pretrained('sentence-transformers/msmarco-distilbert-base-v4', batch_size=100, num_batch_labels=5000, 
                               margin=0.3, num_negatives=5, tau=0.1, apply_softmax=True,
                               
                               data_aug_meta_prefix='cat2data', lbl2data_aug_meta_prefix=None, 
                               data_pred_meta_prefix=None, lbl2data_pred_meta_prefix=None,
                               
                               num_metadata=block.train.dset.meta['cat_meta'].n_meta, resize_length=5000,
                               
                               calib_margin=0.3, calib_num_negatives=10, calib_tau=0.1, calib_apply_softmax=False, 
                               calib_loss_weight=0.1, use_calib_loss=False,

                               use_query_loss=True,

                               meta_loss_weight=0.3, 
                               
                               fusion_loss_weight=0.1, use_fusion_loss=True,
                               
                               use_encoder_parallel=False)

model.init_retrieval_head()
model.init_cross_head()
model.init_meta_embeddings()

Some weights of OAK003 were not initialized from the model checkpoint at sentence-transformers/msmarco-distilbert-base-v4 and are newly initialized: ['encoder.cross_head.k.bias', 'encoder.cross_head.k.weight', 'encoder.cross_head.o.bias', 'encoder.cross_head.o.weight', 'encoder.cross_head.q.bias', 'encoder.cross_head.q.weight', 'encoder.cross_head.v.bias', 'encoder.cross_head.v.weight', 'encoder.dr_fused_head.layer_norm.bias', 'encoder.dr_fused_head.layer_norm.weight', 'encoder.dr_fused_head.projector.bias', 'encoder.dr_fused_head.projector.weight', 'encoder.dr_fused_head.transform.bias', 'encoder.dr_fused_head.transform.weight', 'encoder.dr_head.layer_norm.bias', 'encoder.dr_head.layer_norm.weight', 'encoder.dr_head.projector.bias', 'encoder.dr_head.projector.weight', 'encoder.dr_head.transform.bias', 'encoder.dr_head.transform.weight', 'encoder.meta_embeddings.weight', 'encoder.meta_head.layer_norm.bias', 'encoder.meta_head.layer_norm.weight', 'encoder.meta_head.projector.bias', 'enc

In [ ]:
model.encoder.set_pretrained_meta_embeddings(torch.zeros(656086, 768))
model.encoder.freeze_pretrained_meta_embeddings()

In [ ]:
model = model.to('cuda')

In [ ]:
b = prepare_batch(model, batch, m_args=[
    'plnk2data_idx', 'plnk2data_data2ptr', 'lnk2data_idx', 'lnk2data_input_ids', 'lnk2data_attention_mask', 
    'lnk2data_data2ptr',
])

In [ ]:
b = prepare_batch(model, batch, m_args=[
    'pcat2data_idx', 'pcat2data_data2ptr', 'cat2data_idx', 'cat2data_input_ids', 'cat2data_attention_mask', 
    'cat2data_data2ptr',
])

In [ ]:
o = model(**b.to(model.device))

> /tmp/ipykernel_26893/4002658968.py(65)resize()
     63         #debug
     64 
---> 65         if torch.any(num_inputs == 0): raise ValueError("`num_inputs` should be non-zero positive integer.")
     66         bsz, total_num_inputs = num_inputs.shape[0], idx.shape[0]
     67 



ipdb>  num_inputs


tensor([3, 3, 3, 3, 3])


ipdb>  n


> /tmp/ipykernel_26893/4002658968.py(66)resize()
     64 
     65         if torch.any(num_inputs == 0): raise ValueError("`num_inputs` should be non-zero positive integer.")
---> 66         bsz, total_num_inputs = num_inputs.shape[0], idx.shape[0]
     67 
     68         self.ones = self.ones.to(idx.device)



...

torch.Size([15])


ipdb>  n


> /tmp/ipykernel_26893/3123686291.py(38)fuse_meta_into_embeddings()
     36                 m_repr = F.normalize(self.meta_embeddings(m_idx) + self.pretrained_meta_embeddings(m_idx), dim=1)
     37 
---> 38                 m_repr, m_repr_mask = m_repr.view(len(idx), -1, self.config.dim), m_repr_mask.bool().view(len(idx), -1)
     39                 meta_repr[m_key] = m_repr[m_repr_mask]
     40 



ipdb>  c


/scratch/scai/phd/aiz218323/Projects/xcai/xcai/losses.py:22: UserWarning: Sparse CSR tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at ../aten/src/ATen/SparseCsrTensorImpl.cpp:53.)
  return torch.sparse_csr_tensor(data_ptr, data_idx, scores, device=data_ptr.device)


In [ ]:
o.loss

tensor(0.0675, grad_fn=<AddBackward0>)

## `OAK004`

In [ ]:
#| export
class Encoder004(Encoder003):

    def __init__(
        self, 
        config,
        cross_tau:Optional[float]=0.1, 
        cross_dropout:Optional[float]=0.1, 
        **kwargs
    ):
        super().__init__(config, **kwargs)
        self.cross_head = NormCrossAttention(config, tau=cross_tau, dropout=cross_dropout)
        self.post_init()
    

In [ ]:
#| export
class OAK004(OAK003, DistilBertPreTrainedModel):
    use_generation,use_representation = False,True
    _tied_weights_keys = ["encoder.distilbert"]

    @delegates(OAK003.__init__)
    def __init__(
        self, 
        config,
        num_metadata:int,
        resize_length:Optional[int]=None,
        
        cross_tau:Optional[float]=1.0,
        cross_dropout:Optional[float]=0.1,
        
        **kwargs
    ):
        super().__init__(config, **kwargs, num_metadata=num_metadata, resize_length=resize_length)
        self.encoder = Encoder004(config, cross_tau=cross_tau, cross_dropout=cross_dropout,
                                  num_metadata=num_metadata, resize_length=resize_length)
        self.post_init(); self.remap_post_init(); self.init_retrieval_head(); self.init_cross_head()

    def remap_post_init(self):
        self.distilbert = self.encoder.distilbert

### Example

In [ ]:
model = OAK004.from_pretrained('sentence-transformers/msmarco-distilbert-base-v4', batch_size=100, num_batch_labels=5000, 
                               margin=0.3, num_negatives=5, tau=0.1, apply_softmax=True,
                               
                               data_aug_meta_prefix='lnk2data', lbl2data_aug_meta_prefix=None, 
                               data_pred_meta_prefix=None, lbl2data_pred_meta_prefix=None,
                               
                               num_metadata=block.train.dset.meta['lnk_meta'].n_meta, resize_length=5000,
                               
                               calib_margin=0.05, calib_num_negatives=10, calib_tau=0.1, calib_apply_softmax=False, 
                               calib_loss_weight=0.1, use_calib_loss=False,

                               cross_tau=1.0, cross_dropout=0.1,

                               use_query_loss=True,

                               meta_loss_weight=0.0, 
                               
                               fusion_loss_weight=0.1, use_fusion_loss=False,
                               
                               use_encoder_parallel=False)
model.init_retrieval_head()
model.init_cross_head()

Some weights of OAK004 were not initialized from the model checkpoint at sentence-transformers/msmarco-distilbert-base-v4 and are newly initialized: ['encoder.cross_head.k.bias', 'encoder.cross_head.k.weight', 'encoder.cross_head.o.bias', 'encoder.cross_head.o.weight', 'encoder.cross_head.q.bias', 'encoder.cross_head.q.weight', 'encoder.cross_head.tau', 'encoder.cross_head.v.bias', 'encoder.cross_head.v.weight', 'encoder.dr_fused_head.layer_norm.bias', 'encoder.dr_fused_head.layer_norm.weight', 'encoder.dr_fused_head.projector.bias', 'encoder.dr_fused_head.projector.weight', 'encoder.dr_fused_head.transform.bias', 'encoder.dr_fused_head.transform.weight', 'encoder.dr_head.layer_norm.bias', 'encoder.dr_head.layer_norm.weight', 'encoder.dr_head.projector.bias', 'encoder.dr_head.projector.weight', 'encoder.dr_head.transform.bias', 'encoder.dr_head.transform.weight', 'encoder.meta_embeddings.weight', 'encoder.meta_head.layer_norm.bias', 'encoder.meta_head.layer_norm.weight', 'encoder.meta_

In [ ]:
model.encoder.set_pretrained_meta_embeddings(torch.zeros(656086, 768))
model.encoder.freeze_pretrained_meta_embeddings()

model.init_meta_embeddings()

In [ ]:
model = model.to('cuda')

In [ ]:
b = prepare_batch(model, batch, m_args=[
    'plnk2data_idx', 'plnk2data_data2ptr', 'lnk2data_idx', 'lnk2data_input_ids', 'lnk2data_attention_mask', 
    'lnk2data_data2ptr',
])

In [ ]:
o = model(**b.to(model.device))

In [ ]:
def func():
    import pdb; pdb.set_trace()
    return model(**b.to(model.device))
    

In [ ]:
o.loss

tensor(0.0618, device='cuda:0', grad_fn=<AddBackward0>)

## `OAK005`

In [ ]:
#| export
class Encoder005(Encoder003):

    def __init__(
        self, 
        config,
        **kwargs
    ):
        super().__init__(config, **kwargs)
        self.gen_head = GenerationHead(config)
        
    def get_output_embeddings(self) -> nn.Module:
        return self.gen_head.projector

    def set_output_embeddings(self, new_embeddings: nn.Module):
        self.gen_head.projector = new_embeddings
    
    def gen(self, x:torch.Tensor):
        return self.gen_head(x)

    def forward(
        self, 
        data_input_ids: torch.Tensor, 
        data_attention_mask: torch.Tensor,
        data_aug_meta_prefix: Optional[str]=None,
        data_type:Optional[str]=None,
        data_unnormalized:Optional[bool]=False,

        data_gen_idx:Optional[torch.Tensor]=None,
        
        **kwargs
    ):  
        data_o = self.encode(data_input_ids, data_attention_mask)
        
        if data_type is not None and data_type == "meta":
            data_repr = self.meta_unnormalized(data_o[0], data_attention_mask) if data_unnormalized else self.meta(data_o[0], data_attention_mask)
        else: 
            data_repr = self.dr(data_o[0], data_attention_mask)
        
        data_fused_repr = meta_repr = None
        if data_aug_meta_prefix is not None:
            meta_kwargs = Parameters.from_meta_aug_prefix(data_aug_meta_prefix, **kwargs)
            if len(meta_kwargs):
                data_fused_repr, meta_repr = self.fuse_meta_into_embeddings(data_repr, 
                                                                            torch.any(data_attention_mask, dim=1), 
                                                                            meta_kwargs)
                data_fused_repr = self.dr_fused(data_fused_repr)

        data_logits = self.gen(data_o[0] if data_gen_idx is None else data_o[0][data_gen_idx]) 
                
        return EncoderOutput(
            rep=data_repr,
            fused_rep=data_fused_repr,
            meta_repr=meta_repr,
            
            logits=data_logits,
        )
    

In [ ]:
#| export
class OAK005(OAK003, DistilBertPreTrainedModel):
    use_generation,use_representation = True,True
    _tied_weights_keys = ["encoder.distilbert"]

    @delegates(OAK003.__init__)
    def __init__(
        self, 
        config,
        num_metadata:int,
        resize_length:Optional[int]=None,

        num_batch_labels:Optional[int]=None, 
        ignore_token:Optional[int]=0,
        gen_loss_weight:Optional[float]=1.0,
        use_gen_loss:Optional[bool]=True,
        
        **kwargs
    ):
        super().__init__(config, **kwargs, num_metadata=num_metadata, resize_length=resize_length, num_batch_labels=num_batch_labels)
        store_attr('use_gen_loss')
        self.g_lw = gen_loss_weight
        self.encoder = Encoder005(config, num_metadata=num_metadata, resize_length=resize_length)

        self.gen_loss_fn = MultiCrossEntropy(tn_targ=num_batch_labels, ig_tok=ignore_token, reduce='mean')
        self.post_init(); self.remap_post_init(); self.init_retrieval_head(); self.init_cross_head()

    def remap_post_init(self):
        self.distilbert = self.encoder.distilbert

    def compute_gen_loss(self, inp_logits, targ_logits, inp_input_ids, targ_input_ids, targ_ptr):
        gen_loss = self.gen_loss_fn(inp_logits, targ_input_ids, targ_ptr) + self.gen_loss_fn(targ_logits, inp_input_ids)
        return self.g_lw * gen_loss

    def init_generation_head(self):
        self.encoder.gen_head.projector.weight.data = self.get_input_embeddings().weight.data.clone()

    def get_last_item_mask(self, num_input:torch.Tensor, input_sz:int):
        idx = torch.where(num_input > 0)[0]
        input_ptr = num_input[idx].cumsum(dim=0)-1
        return torch.zeros(input_sz, dtype=torch.bool, device=num_input.device).scatter(0, input_ptr, 1)

    def freeze(self):
        for n,p in self.named_parameters():
            p.requires_grad_(False)

    def unfreeze(self):
        for n,p in self.named_parameters():
            p.requires_grad_(True)

    def unfreeze_head(self):
        for n,p in self.encoder.gen_head.named_parameters():
            p.requires_grad_(True)

    def forward(
        self,
        data_input_ids:Optional[torch.Tensor]=None,
        data_attention_mask:Optional[torch.Tensor]=None,
        lbl2data_data2ptr:Optional[torch.Tensor]=None,
        lbl2data_idx:Optional[torch.Tensor]=None,
        lbl2data_input_ids:Optional[torch.Tensor]=None,
        lbl2data_attention_mask:Optional[torch.Tensor]=None,
        plbl2data_data2ptr:Optional[torch.Tensor]=None,
        plbl2data_idx:Optional[torch.Tensor]=None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
        **kwargs
    ):  
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict
        
        if self.use_encoder_parallel: 
            encoder = XCDataParallel(module=self.encoder)
        else: encoder = self.encoder
        
        data_meta_kwargs = Parameters.from_feat_meta_aug_prefix('data', self.data_aug_meta_prefix, **kwargs)
        data_o = encoder(data_input_ids=data_input_ids, data_attention_mask=data_attention_mask, 
                         data_aug_meta_prefix=self.data_aug_meta_prefix, **data_meta_kwargs)
        
        
        loss = None; lbl2data_o = EncoderOutput()
        if lbl2data_input_ids is not None:
            lbl2data_gen_idx = self.get_last_item_mask(lbl2data_data2ptr, len(lbl2data_idx))
            lbl2data_meta_kwargs = Parameters.from_feat_meta_aug_prefix('lbl2data', self.lbl2data_aug_meta_prefix, **kwargs)
            
            lbl2data_o = encoder(data_input_ids=lbl2data_input_ids, data_attention_mask=lbl2data_attention_mask, 
                                 data_aug_meta_prefix=self.lbl2data_aug_meta_prefix, data_gen_idx=lbl2data_gen_idx,
                                 **lbl2data_meta_kwargs)
            
            loss = self.compute_loss(data_o.fused_rep, lbl2data_o.rep,lbl2data_data2ptr,lbl2data_idx,
                                     plbl2data_data2ptr,plbl2data_idx)

            if self.use_gen_loss:
                loss += self.compute_gen_loss(data_o.logits, lbl2data_o.logits, data_input_ids,lbl2data_input_ids,
                                              lbl2data_data2ptr)

            if self.use_query_loss:
                loss += self.compute_loss(data_o.rep, lbl2data_o.rep,lbl2data_data2ptr,lbl2data_idx,
                                          plbl2data_data2ptr,plbl2data_idx)

            if self.use_calib_loss:
                loss += self.calibration_loss(data_o.fused_rep, data_o.rep, lbl2data_o.rep,lbl2data_data2ptr,lbl2data_idx,
                                              plbl2data_data2ptr,plbl2data_idx)
            
            loss += self.compute_meta_loss(data_o.fused_rep, lbl2data_o.rep, **kwargs)
            
            if self.use_fusion_loss:
                loss += self.compute_fusion_loss(data_o.fused_rep, data_o.meta_repr, self.data_aug_meta_prefix, **kwargs)
                loss += self.compute_fusion_loss(lbl2data_o.rep, lbl2data_o.meta_repr, self.lbl2data_aug_meta_prefix, **kwargs)
            
            
        if not return_dict:
            o = (data_o.logits,data_o.rep,data_o.fused_rep,lbl2data_o.logits,lbl2data_o.rep,lbl2data_o.fused_rep)
            return ((loss,) + o) if loss is not None else o
        
        
        return XCModelOutput(
            loss=loss,
            
            data_repr=data_o.rep,
            data_fused_repr=data_o.fused_rep,

            logits=data_o.logits,
            
            lbl2data_repr=lbl2data_o.rep,
            lbl2data_fused_repr=lbl2data_o.fused_rep,
        )
    

### Example

In [ ]:
model = OAK005.from_pretrained('sentence-transformers/msmarco-distilbert-base-v4', batch_size=100, num_batch_labels=5000, 
                               margin=0.3, num_negatives=5, tau=0.1, apply_softmax=True,
                               
                               data_aug_meta_prefix='lnk2data', lbl2data_aug_meta_prefix=None, 
                               data_pred_meta_prefix=None, lbl2data_pred_meta_prefix=None,
                               
                               num_metadata=block.train.dset.meta['lnk_meta'].n_meta, resize_length=5000,
                               
                               calib_margin=0.05, calib_num_negatives=10, calib_tau=0.1, calib_apply_softmax=False, 
                               calib_loss_weight=0.1, use_calib_loss=False,
                               
                               use_query_loss=True,

                               meta_loss_weight=0.0, 
                               
                               fusion_loss_weight=0.1, use_fusion_loss=False,

                               ignore_token=0, gen_loss_weight=1.0, use_gen_loss=True,
                               
                               use_encoder_parallel=False)

model.init_generation_head()

Some weights of OAK005 were not initialized from the model checkpoint at sentence-transformers/msmarco-distilbert-base-v4 and are newly initialized: ['encoder.cross_head.k.bias', 'encoder.cross_head.k.weight', 'encoder.cross_head.o.bias', 'encoder.cross_head.o.weight', 'encoder.cross_head.q.bias', 'encoder.cross_head.q.weight', 'encoder.cross_head.v.bias', 'encoder.cross_head.v.weight', 'encoder.dr_fused_head.layer_norm.bias', 'encoder.dr_fused_head.layer_norm.weight', 'encoder.dr_fused_head.projector.bias', 'encoder.dr_fused_head.projector.weight', 'encoder.dr_fused_head.transform.bias', 'encoder.dr_fused_head.transform.weight', 'encoder.dr_head.layer_norm.bias', 'encoder.dr_head.layer_norm.weight', 'encoder.dr_head.projector.bias', 'encoder.dr_head.projector.weight', 'encoder.dr_head.transform.bias', 'encoder.dr_head.transform.weight', 'encoder.gen_head.layer_norm.bias', 'encoder.gen_head.layer_norm.weight', 'encoder.gen_head.projector.bias', 'encoder.gen_head.projector.weight', 'enc

In [ ]:
model.freeze()
model.unfreeze_head()

In [ ]:
model.init_retrieval_head()
model.init_cross_head()
model.init_meta_embeddings()

model.encoder.set_pretrained_meta_embeddings(torch.zeros(656086, 768))
model.encoder.freeze_pretrained_meta_embeddings()

In [ ]:
model = model.to('cuda')

In [ ]:
b = prepare_batch(model, batch, m_args=[
    'plnk2data_idx', 'plnk2data_data2ptr', 'lnk2data_idx', 'lnk2data_input_ids', 'lnk2data_attention_mask', 
    'lnk2data_data2ptr',
])

In [ ]:
o = model(**b.to(model.device))

> /tmp/ipykernel_3071/2330200620.py(58)forward()
     56         **kwargs
     57     ):  
---> 58         return_dict = return_dict if return_dict is not None else self.config.use_return_dict
     59 
     60         if self.use_encoder_parallel:



ipdb>  c


> /tmp/ipykernel_3071/1501785349.py(33)forward()
     31         **kwargs
     32     ):  
---> 33         data_o = self.encode(data_input_ids, data_attention_mask)
     34 
     35         if data_type is not None and data_type == "meta":



ipdb>  c


> /tmp/ipykernel_3071/1501785349.py(33)forward()
     31         **kwargs
     32     ):  
---> 33         data_o = self.encode(data_input_ids, data_attention_mask)
     34 
     35         if data_type is not None and data_type == "meta":



ipdb>  c


In [ ]:
def func():
    import pdb; pdb.set_trace()
    return model(**b.to(model.device))
    

In [ ]:
o = func()

> /tmp/ipykernel_3071/3657616883.py(3)func()
      1 def func():
      2     import pdb; pdb.set_trace()
----> 3     return model(**b.to(model.device))
      4 



ipdb>  b model.forward


Breakpoint 1 at /tmp/ipykernel_3071/2330200620.py:43


ipdb>  b model.encoder.forward


Breakpoint 2 at /tmp/ipykernel_3071/1501785349.py:21


...

--Return--
XCModelOutput...sed_repr=None)
> /tmp/ipykernel_3071/3657616883.py(3)func()
      1 def func():
      2     import pdb; pdb.set_trace()
----> 3     return model(**b.to(model.device))
      4 



ipdb>  


--Return--
None
> /tmp/ipykernel_3071/492731717.py(1)<module>()
----> 1 o = func()



ipdb>  c


    [... skipped 1 hidden frame]

    [... skipped 1 hidden frame]

    [... skipped 1 hidden frame]

    [... skipped 1 hidden frame]



In [ ]:
o.loss

tensor(21.9783, grad_fn=<AddBackward0>)

## `OAK006`

In [ ]:
#| export
class Encoder006(DistilBertPreTrainedModel):
    
    def __init__(
        self, 
        config:PretrainedConfig, 
        num_metadata:int,
        resize_length:Optional[int]=None,
    ):
        super().__init__(config)
        self.distilbert = DistilBertModel(config)
        
        self.dr_head = RepresentationHead(config)
        self.dr_fused_head = RepresentationHead(config)
        self.meta_head = RepresentationHead(config)
        self.cross_head = CrossAttention(config)
        self.meta_embeddings = nn.Embedding(num_metadata, config.dim, sparse=True)
        self.pretrained_meta_embeddings = nn.Embedding(num_metadata, config.dim)

        self.ones = torch.ones(resize_length, dtype=torch.long, device=self.device) if resize_length is not None else None
        self.post_init()

    def freeze_meta_embeddings(self):
        self.meta_embeddings.requires_grad_(False)

    def unfreeze_meta_embeddings(self):
        self.meta_embeddings.requires_grad_(True)

    def set_meta_embeddings(self, embed:torch.Tensor):
        self.meta_embeddings.weight.data = embed

    def init_meta_embeddings(self):
        self.meta_embeddings.weight.data = torch.zeros_like(self.meta_embeddings.weight.data)

    def freeze_pretrained_meta_embeddings(self):
        self.pretrained_meta_embeddings.requires_grad_(False)

    def unfreeze_pretrained_meta_embeddings(self):
        self.pretrained_meta_embeddings.requires_grad_(True)

    def set_pretrained_meta_embeddings(self, embed:torch.Tensor):
        self.pretrained_meta_embeddings.weight.data = embed
        
    def get_position_embeddings(self) -> nn.Embedding:
        return self.distilbert.get_position_embeddings()
    
    def resize_position_embeddings(self, new_num_position_embeddings: int):
        self.distilbert.resize_position_embeddings(new_num_position_embeddings)
    
    def encode(self, input_ids:torch.Tensor, attention_mask:torch.Tensor, **kwargs):
        return self.distilbert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            **kwargs
        )
    
    def dr(self, embed:torch.Tensor, attention_mask:torch.Tensor):
        embed = self.dr_head(embed)
        return F.normalize(Pooling.mean_pooling(embed, attention_mask), dim=1)

    def dr_fused(self, embed:torch.Tensor):
        embed = self.dr_fused_head(embed)
        return F.normalize(embed, dim=1)

    def meta(self, embed:torch.Tensor, attention_mask:torch.Tensor):
        embed = self.meta_head(embed)
        return F.normalize(Pooling.mean_pooling(embed, attention_mask), dim=1)
    
    def meta_unnormalized(self, embed:torch.Tensor, attention_mask:torch.Tensor):
        embed = self.meta_head(embed)
        return Pooling.mean_pooling(embed, attention_mask)

    def resize(self, idx:torch.Tensor, num_inputs:torch.Tensor):
        if torch.any(num_inputs == 0): raise ValueError("`num_inputs` should be non-zero positive integer.")
        bsz, total_num_inputs = num_inputs.shape[0], idx.shape[0]
        
        self.ones = self.ones.to(idx.device)
        ones = (
            torch.ones(total_num_inputs, dtype=torch.long, device=idx.device) 
            if self.ones is None or self.ones.shape[0] < total_num_inputs else self.ones[:total_num_inputs]
        )

        max_num_inputs = num_inputs.max()
        if (num_inputs == max_num_inputs).all():
            return idx,ones
        
        xnum_inputs = max_num_inputs-num_inputs+1

        inputs_ptr = num_inputs.cumsum(dim=0)-1
        repeat_inputs = ones.scatter(0, inputs_ptr, xnum_inputs)
        
        resized_idx = idx.repeat_interleave(repeat_inputs, dim=0)
        ignore_mask = ones.scatter(0, inputs_ptr, 0).repeat_interleave(repeat_inputs, dim=0).view(bsz, -1)
        ignore_mask[:, -1] = 1; ignore_mask = ignore_mask.flatten()
        
        return resized_idx,ignore_mask

    def fuse_meta_into_embeddings(self, data_repr:torch.Tensor, data_mask:torch.Tensor, meta_kwargs:Dict):
        meta_repr = {}
        
        data_fused_repr, data_mask = data_repr.clone().view(-1, 1, self.config.dim), data_mask.view(-1, 1)
        for m_key, m_args in meta_kwargs.items():
            idx = torch.where(m_args['data2ptr'] > 0)[0]
            meta_repr[m_key] = torch.empty(0, self.config.dim).to(data_repr)
            
            if len(idx):
                m_idx,m_repr_mask = self.resize(m_args['idx'], m_args['data2ptr'][idx])
                m_repr = F.normalize(self.meta_embeddings(m_idx) + self.pretrained_meta_embeddings(m_idx), dim=1)
                
                m_repr, m_repr_mask = m_repr.view(len(idx), -1, self.config.dim), m_repr_mask.bool().view(len(idx), -1)
                meta_repr[m_key] = m_repr[m_repr_mask]
                
                fused_repr = self.cross_head(data_fused_repr[idx], data_mask[idx], m_repr, m_repr_mask)[0]
                data_fused_repr[idx] += fused_repr
                
        return data_fused_repr.squeeze(), meta_repr

    def forward(
        self, 
        data_input_ids: torch.Tensor, 
        data_attention_mask: torch.Tensor,
        data_aug_meta_prefix: Optional[str]=None,
        data_type:Optional[str]=None,
        data_unnormalized:Optional[bool]=False,
        **kwargs
    ):  
        data_o = self.encode(data_input_ids, data_attention_mask)
        
        if data_type is not None and data_type == "meta":
            data_repr = self.meta_unnormalized(data_o[0], data_attention_mask) if data_unnormalized else self.meta(data_o[0], data_attention_mask)
        else: 
            data_repr = self.dr(data_o[0], data_attention_mask)
        
        data_fused_repr = meta_repr = None
        if data_aug_meta_prefix is not None:
            meta_kwargs = Parameters.from_meta_aug_prefix(data_aug_meta_prefix, **kwargs)
            if len(meta_kwargs):
                data_fused_repr, meta_repr = self.fuse_meta_into_embeddings(data_repr, 
                                                                            torch.any(data_attention_mask, dim=1), 
                                                                            meta_kwargs)
                data_fused_repr = self.dr_fused(data_fused_repr)
                
        return EncoderOutput(
            rep=data_repr,
            fused_rep=data_fused_repr,
            meta_repr=meta_repr,
        )
        

In [ ]:
#| export
class OAK006(OAK000, DistilBertPreTrainedModel):
    use_generation,use_representation = False,True
    _tied_weights_keys = ["encoder.distilbert"]

    @delegates(OAK000.__init__)
    def __init__(
        self, 
        config,
        num_metadata:int,
        resize_length:Optional[int]=None,
        **kwargs
    ):
        super().__init__(config, **kwargs)
        self.encoder = Encoder006(config, num_metadata=num_metadata, resize_length=resize_length)
        self.post_init(); self.remap_post_init(); self.init_retrieval_head(); self.init_cross_head()

    def init_meta_embeddings(self):
        self.encoder.init_meta_embeddings()

    def remap_post_init(self):
        self.distilbert = self.encoder.distilbert
        

### Example

In [ ]:
model = OAK006.from_pretrained('sentence-transformers/msmarco-distilbert-base-v4', batch_size=100, num_batch_labels=5000, 
                               margin=0.3, num_negatives=5, tau=0.1, apply_softmax=True,
                               
                               data_aug_meta_prefix='cat2data', lbl2data_aug_meta_prefix=None, 
                               data_pred_meta_prefix=None, lbl2data_pred_meta_prefix=None,
                               
                               num_metadata=block.train.dset.meta['cat_meta'].n_meta, resize_length=5000,
                               
                               calib_margin=0.3, calib_num_negatives=10, calib_tau=0.1, calib_apply_softmax=False, 
                               calib_loss_weight=0.1, use_calib_loss=False,

                               use_query_loss=True,

                               meta_loss_weight=0.0, 
                               
                               fusion_loss_weight=0.1, use_fusion_loss=False,
                               
                               use_encoder_parallel=False)

model.init_retrieval_head()
model.init_cross_head()
model.init_meta_embeddings()

Some weights of OAK006 were not initialized from the model checkpoint at sentence-transformers/msmarco-distilbert-base-v4 and are newly initialized: ['encoder.cross_head.k.bias', 'encoder.cross_head.k.weight', 'encoder.cross_head.o.bias', 'encoder.cross_head.o.weight', 'encoder.cross_head.q.bias', 'encoder.cross_head.q.weight', 'encoder.cross_head.v.bias', 'encoder.cross_head.v.weight', 'encoder.dr_fused_head.layer_norm.bias', 'encoder.dr_fused_head.layer_norm.weight', 'encoder.dr_fused_head.projector.bias', 'encoder.dr_fused_head.projector.weight', 'encoder.dr_fused_head.transform.bias', 'encoder.dr_fused_head.transform.weight', 'encoder.dr_head.layer_norm.bias', 'encoder.dr_head.layer_norm.weight', 'encoder.dr_head.projector.bias', 'encoder.dr_head.projector.weight', 'encoder.dr_head.transform.bias', 'encoder.dr_head.transform.weight', 'encoder.meta_embeddings.weight', 'encoder.meta_head.layer_norm.bias', 'encoder.meta_head.layer_norm.weight', 'encoder.meta_head.projector.bias', 'enc

In [ ]:
model.encoder.set_pretrained_meta_embeddings(torch.zeros(656086, 768))
model.encoder.freeze_pretrained_meta_embeddings()

In [ ]:
model = model.to('cuda')

In [ ]:
b = prepare_batch(model, batch, m_args=[
    'pcat2data_idx', 'pcat2data_data2ptr', 'cat2data_idx', 'cat2data_input_ids', 'cat2data_attention_mask', 
    'cat2data_data2ptr',
])

In [ ]:
o = model(**b.to(model.device))

In [ ]:
o.loss

tensor(0.0160, device='cuda:0', grad_fn=<AddBackward0>)

## `OAK007`

In [20]:
#| export
class OAK007(OAK003, DistilBertPreTrainedModel):
    
    @delegates(OAK003.__init__)
    def __init__(
        self, 
        config,
        n_labels:int,
        **kwargs
    ):
        super().__init__(config, **kwargs)
        self.label_embeddings = nn.Embedding(n_labels, config.dim)
        self.post_init(); self.remap_post_init(); self.init_retrieval_head(); self.init_cross_head()

    def init_label_embeddings(self):
        self.label_embeddings.weight.data = torch.zeros_like(self.label_embeddings.weight.data)

    def get_label_representation(
        self,
        data_idx:Optional[torch.Tensor]=None,
        data_input_ids:Optional[torch.Tensor]=None,
        data_attention_mask:Optional[torch.Tensor]=None,
        **kwargs
    ):
        if self.use_encoder_parallel: 
            encoder = XCDataParallel(module=self.encoder)
        else: encoder = self.encoder
        data_o = encoder(data_input_ids=data_input_ids, data_attention_mask=data_attention_mask)
        data_o.rep = F.normalize(data_o.rep + self.label_embeddings(data_idx), dim=1)
        return XCModelOutput(
            data_repr=data_o.rep,
            data_fused_repr=data_o.fused_rep,
        )

    def forward(
        self,
        data_idx:Optional[torch.Tensor]=None,
        data_input_ids:Optional[torch.Tensor]=None,
        data_attention_mask:Optional[torch.Tensor]=None,
        lbl2data_data2ptr:Optional[torch.Tensor]=None,
        lbl2data_idx:Optional[torch.Tensor]=None,
        lbl2data_input_ids:Optional[torch.Tensor]=None,
        lbl2data_attention_mask:Optional[torch.Tensor]=None,
        plbl2data_data2ptr:Optional[torch.Tensor]=None,
        plbl2data_idx:Optional[torch.Tensor]=None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
        **kwargs
    ):  
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict
        
        if self.use_encoder_parallel: 
            encoder = XCDataParallel(module=self.encoder)
        else: encoder = self.encoder
        
        data_meta_kwargs = Parameters.from_feat_meta_aug_prefix('data', self.data_aug_meta_prefix, **kwargs)
        data_o = encoder(data_input_ids=data_input_ids, data_attention_mask=data_attention_mask, 
                         data_aug_meta_prefix=self.data_aug_meta_prefix, **data_meta_kwargs)

        loss = None; lbl2data_o = EncoderOutput()
        if lbl2data_input_ids is not None:
            lbl2data_meta_kwargs = Parameters.from_feat_meta_aug_prefix('lbl2data', self.lbl2data_aug_meta_prefix, **kwargs)
            lbl2data_o = encoder(data_input_ids=lbl2data_input_ids, data_attention_mask=lbl2data_attention_mask, 
                                 data_aug_meta_prefix=self.lbl2data_aug_meta_prefix, **lbl2data_meta_kwargs)
            lbl2data_o.rep = F.normalize(lbl2data_o.rep + self.label_embeddings(lbl2data_idx), dim=1)
            
            loss = self.compute_loss(data_o.fused_rep, lbl2data_o.rep,lbl2data_data2ptr,lbl2data_idx,
                                     plbl2data_data2ptr,plbl2data_idx)

            if self.use_query_loss:
                loss += self.compute_loss(data_o.rep, lbl2data_o.rep,lbl2data_data2ptr,lbl2data_idx,
                                          plbl2data_data2ptr,plbl2data_idx)

            if self.use_calib_loss:
                loss += self.calibration_loss(data_o.fused_rep, data_o.rep, lbl2data_o.rep,lbl2data_data2ptr,lbl2data_idx,
                                              plbl2data_data2ptr,plbl2data_idx)
            
            loss += self.compute_meta_loss(data_o.fused_rep, lbl2data_o.rep, **kwargs)
            
            if self.use_fusion_loss:
                loss += self.compute_fusion_loss(data_o.fused_rep, data_o.meta_repr, self.data_aug_meta_prefix, **kwargs)
                loss += self.compute_fusion_loss(lbl2data_o.rep, lbl2data_o.meta_repr, self.lbl2data_aug_meta_prefix, **kwargs)
            
            
        if not return_dict:
            o = (data_o.logits,data_o.rep,data_o.fused_rep,lbl2data_o.logits,lbl2data_o.rep,lbl2data_o.fused_rep)
            return ((loss,) + o) if loss is not None else o

        return XCModelOutput(
            loss=loss,
            
            data_repr=data_o.rep,
            data_fused_repr=data_o.fused_rep,
            
            lbl2data_repr=lbl2data_o.rep,
            lbl2data_fused_repr=lbl2data_o.fused_rep,
        )
        

### Example

In [22]:
model = OAK007.from_pretrained('sentence-transformers/msmarco-distilbert-base-v4', batch_size=100, num_batch_labels=5000, 
                               margin=0.3, num_negatives=5, tau=0.1, apply_softmax=True,
                               
                               data_aug_meta_prefix='cat2data', lbl2data_aug_meta_prefix=None, 
                               data_pred_meta_prefix=None, lbl2data_pred_meta_prefix=None,
                               
                               num_metadata=block.train.dset.meta['cat_meta'].n_meta, resize_length=5000, n_labels=block.n_lbl,
                               
                               calib_margin=0.3, calib_num_negatives=10, calib_tau=0.1, calib_apply_softmax=False, 
                               calib_loss_weight=0.1, use_calib_loss=False,

                               use_query_loss=True,

                               meta_loss_weight=0.0, 
                               
                               fusion_loss_weight=0.1, use_fusion_loss=False,
                               
                               use_encoder_parallel=False)

model.init_retrieval_head()
model.init_cross_head()
model.init_meta_embeddings()
model.init_label_embeddings()

Some weights of OAK007 were not initialized from the model checkpoint at sentence-transformers/msmarco-distilbert-base-v4 and are newly initialized: ['encoder.cross_head.k.bias', 'encoder.cross_head.k.weight', 'encoder.cross_head.o.bias', 'encoder.cross_head.o.weight', 'encoder.cross_head.q.bias', 'encoder.cross_head.q.weight', 'encoder.cross_head.v.bias', 'encoder.cross_head.v.weight', 'encoder.dr_fused_head.layer_norm.bias', 'encoder.dr_fused_head.layer_norm.weight', 'encoder.dr_fused_head.projector.bias', 'encoder.dr_fused_head.projector.weight', 'encoder.dr_fused_head.transform.bias', 'encoder.dr_fused_head.transform.weight', 'encoder.dr_head.layer_norm.bias', 'encoder.dr_head.layer_norm.weight', 'encoder.dr_head.projector.bias', 'encoder.dr_head.projector.weight', 'encoder.dr_head.transform.bias', 'encoder.dr_head.transform.weight', 'encoder.meta_embeddings.weight', 'encoder.meta_head.layer_norm.bias', 'encoder.meta_head.layer_norm.weight', 'encoder.meta_head.projector.bias', 'enc

In [ ]:
model.encoder.set_pretrained_meta_embeddings(torch.zeros(656086, 768))
model.encoder.freeze_pretrained_meta_embeddings()

In [ ]:
model = model.to('cuda')

In [ ]:
b = prepare_batch(model, batch, m_args=[
    'pcat2data_idx', 'pcat2data_data2ptr', 'cat2data_idx', 'cat2data_input_ids', 'cat2data_attention_mask', 
    'cat2data_data2ptr',
])

In [30]:
o = model(**b.to(model.device))

In [31]:
o.loss

tensor(0.0609, device='cuda:0', grad_fn=<AddBackward0>)

## `OAK008`

In [17]:
#| export
class OAK008(OAK003, DistilBertPreTrainedModel):
    
    @delegates(OAK003.__init__)
    def __init__(
        self, 
        config,
        n_labels:int,
        n_clusters:int,
        **kwargs
    ):
        super().__init__(config, **kwargs)
        self.label_embeddings = nn.Embedding(n_clusters, config.dim)
        self.register_buffer("label_remap", torch.arange(n_labels)%n_clusters, persistent=True)
        self.post_init(); self.remap_post_init(); self.init_retrieval_head(); self.init_cross_head()

    def init_label_embeddings(self):
        self.label_embeddings.weight.data = torch.zeros_like(self.label_embeddings.weight.data)

    def set_label_embeddings(self, embed:torch.Tensor):
        self.label_embeddings.weight.data = embed

    def set_label_remap(self, label_remap:torch.Tensor):
        if label_remap.shape[0] != self.label_remap.shape[0]:
            raise ValueError(f'Shape mismatch, `label_remap` should have {self.label_remap.shape[0]} elements.')
        self.label_remap = label_remap

    def get_label_representation(
        self,
        data_idx:Optional[torch.Tensor]=None,
        data_input_ids:Optional[torch.Tensor]=None,
        data_attention_mask:Optional[torch.Tensor]=None,
        **kwargs
    ):
        if self.use_encoder_parallel: 
            encoder = XCDataParallel(module=self.encoder)
        else: encoder = self.encoder
        data_o = encoder(data_input_ids=data_input_ids, data_attention_mask=data_attention_mask)
        data_o.rep = F.normalize(data_o.rep + self.label_embeddings(self.label_remap[data_idx]), dim=1)
        return XCModelOutput(
            data_repr=data_o.rep,
            data_fused_repr=data_o.fused_rep,
        )

    def forward(
        self,
        data_idx:Optional[torch.Tensor]=None,
        data_input_ids:Optional[torch.Tensor]=None,
        data_attention_mask:Optional[torch.Tensor]=None,
        lbl2data_data2ptr:Optional[torch.Tensor]=None,
        lbl2data_idx:Optional[torch.Tensor]=None,
        lbl2data_input_ids:Optional[torch.Tensor]=None,
        lbl2data_attention_mask:Optional[torch.Tensor]=None,
        plbl2data_data2ptr:Optional[torch.Tensor]=None,
        plbl2data_idx:Optional[torch.Tensor]=None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
        **kwargs
    ):  
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict
        
        if self.use_encoder_parallel: 
            encoder = XCDataParallel(module=self.encoder)
        else: encoder = self.encoder
        
        data_meta_kwargs = Parameters.from_feat_meta_aug_prefix('data', self.data_aug_meta_prefix, **kwargs)
        data_o = encoder(data_input_ids=data_input_ids, data_attention_mask=data_attention_mask, 
                         data_aug_meta_prefix=self.data_aug_meta_prefix, **data_meta_kwargs)

        loss = None; lbl2data_o = EncoderOutput()
        if lbl2data_input_ids is not None:
            lbl2data_meta_kwargs = Parameters.from_feat_meta_aug_prefix('lbl2data', self.lbl2data_aug_meta_prefix, **kwargs)
            lbl2data_o = encoder(data_input_ids=lbl2data_input_ids, data_attention_mask=lbl2data_attention_mask, 
                                 data_aug_meta_prefix=self.lbl2data_aug_meta_prefix, **lbl2data_meta_kwargs)
            lbl2data_o.rep = F.normalize(lbl2data_o.rep + self.label_embeddings(self.label_remap[lbl2data_idx]), dim=1)
            
            loss = self.compute_loss(data_o.fused_rep, lbl2data_o.rep,lbl2data_data2ptr,lbl2data_idx,
                                     plbl2data_data2ptr,plbl2data_idx)

            if self.use_query_loss:
                loss += self.compute_loss(data_o.rep, lbl2data_o.rep,lbl2data_data2ptr,lbl2data_idx,
                                          plbl2data_data2ptr,plbl2data_idx)

            if self.use_calib_loss:
                loss += self.calibration_loss(data_o.fused_rep, data_o.rep, lbl2data_o.rep,lbl2data_data2ptr,lbl2data_idx,
                                              plbl2data_data2ptr,plbl2data_idx)
            
            loss += self.compute_meta_loss(data_o.fused_rep, lbl2data_o.rep, **kwargs)
            
            if self.use_fusion_loss:
                loss += self.compute_fusion_loss(data_o.fused_rep, data_o.meta_repr, self.data_aug_meta_prefix, **kwargs)
                loss += self.compute_fusion_loss(lbl2data_o.rep, lbl2data_o.meta_repr, self.lbl2data_aug_meta_prefix, **kwargs)
            
            
        if not return_dict:
            o = (data_o.logits,data_o.rep,data_o.fused_rep,lbl2data_o.logits,lbl2data_o.rep,lbl2data_o.fused_rep)
            return ((loss,) + o) if loss is not None else o

        return XCModelOutput(
            loss=loss,
            
            data_repr=data_o.rep,
            data_fused_repr=data_o.fused_rep,
            
            lbl2data_repr=lbl2data_o.rep,
            lbl2data_fused_repr=lbl2data_o.fused_rep,
        )
        

### Example

In [20]:
model = OAK008.from_pretrained('sentence-transformers/msmarco-distilbert-base-v4', batch_size=100, num_batch_labels=5000, 
                               margin=0.3, num_negatives=5, tau=0.1, apply_softmax=True,
                               
                               data_aug_meta_prefix='cat2data', lbl2data_aug_meta_prefix=None, 
                               data_pred_meta_prefix=None, lbl2data_pred_meta_prefix=None,
                               
                               num_metadata=block.train.dset.meta['cat_meta'].n_meta, resize_length=5000, 
                               n_labels=block.n_lbl, n_clusters=block.n_lbl//3,
                               
                               calib_margin=0.3, calib_num_negatives=10, calib_tau=0.1, calib_apply_softmax=False, 
                               calib_loss_weight=0.1, use_calib_loss=False,

                               use_query_loss=True,

                               meta_loss_weight=0.0, 
                               
                               fusion_loss_weight=0.1, use_fusion_loss=False,
                               
                               use_encoder_parallel=False)

model.init_retrieval_head()
model.init_cross_head()
model.init_meta_embeddings()
model.init_label_embeddings()

Some weights of OAK008 were not initialized from the model checkpoint at sentence-transformers/msmarco-distilbert-base-v4 and are newly initialized: ['encoder.cross_head.k.bias', 'encoder.cross_head.k.weight', 'encoder.cross_head.o.bias', 'encoder.cross_head.o.weight', 'encoder.cross_head.q.bias', 'encoder.cross_head.q.weight', 'encoder.cross_head.v.bias', 'encoder.cross_head.v.weight', 'encoder.dr_fused_head.layer_norm.bias', 'encoder.dr_fused_head.layer_norm.weight', 'encoder.dr_fused_head.projector.bias', 'encoder.dr_fused_head.projector.weight', 'encoder.dr_fused_head.transform.bias', 'encoder.dr_fused_head.transform.weight', 'encoder.dr_head.layer_norm.bias', 'encoder.dr_head.layer_norm.weight', 'encoder.dr_head.projector.bias', 'encoder.dr_head.projector.weight', 'encoder.dr_head.transform.bias', 'encoder.dr_head.transform.weight', 'encoder.meta_embeddings.weight', 'encoder.meta_head.layer_norm.bias', 'encoder.meta_head.layer_norm.weight', 'encoder.meta_head.projector.bias', 'enc

In [21]:
model.encoder.set_pretrained_meta_embeddings(torch.zeros(656086, 768))
model.encoder.freeze_pretrained_meta_embeddings()

In [22]:
model = model.to('cuda')

In [23]:
b = prepare_batch(model, batch, m_args=[
    'pcat2data_idx', 'pcat2data_data2ptr', 'cat2data_idx', 'cat2data_input_ids', 'cat2data_attention_mask', 
    'cat2data_data2ptr',
])

In [24]:
o = model(**b.to(model.device))

/scratch/scai/phd/aiz218323/Projects/xcai/xcai/losses.py:22: UserWarning: Sparse CSR tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at ../aten/src/ATen/SparseCsrTensorImpl.cpp:53.)
  return torch.sparse_csr_tensor(data_ptr, data_idx, scores, device=data_ptr.device)


In [25]:
o.loss

tensor(0.0260, device='cuda:0', grad_fn=<AddBackward0>)

## `OAK009`

In [17]:
#| export
class OAK009(OAK008, DistilBertPreTrainedModel):
    
    @delegates(OAK008.__init__)
    def __init__(
        self, 
        config,
        embed_dim:int,
        **kwargs
    ):
        super().__init__(config, **kwargs)
        self.transform = nn.Linear(config.dim, embed_dim)
        self.post_init(); self.remap_post_init(); self.init_retrieval_head(); self.init_cross_head()

    def init_transform(self):
        self.transform.weight.data = torch.eye(self.transform.out_features, self.transform.in_features, 
                                               dtype=self.transform.weight.dtype)

    def get_label_representation(
        self,
        data_idx:Optional[torch.Tensor]=None,
        data_input_ids:Optional[torch.Tensor]=None,
        data_attention_mask:Optional[torch.Tensor]=None,
        **kwargs
    ):
        if self.use_encoder_parallel: 
            encoder = XCDataParallel(module=self.encoder)
        else: encoder = self.encoder
        data_o = encoder(data_input_ids=data_input_ids, data_attention_mask=data_attention_mask)
        data_o.rep = F.normalize(data_o.rep + self.label_embeddings(self.label_remap[data_idx]), dim=1)

        if data_o.rep is not None: data_o.rep = F.normalize(self.transform(data_o.rep), dim=1)
        if data_o.fused_rep is not None: data_o.fused_rep = F.normalize(self.transform(data_o.fused_rep), dim=1)
        
        return XCModelOutput(
            data_repr=data_o.rep,
            data_fused_repr=data_o.fused_rep,
        )

    def forward(
        self,
        data_idx:Optional[torch.Tensor]=None,
        data_input_ids:Optional[torch.Tensor]=None,
        data_attention_mask:Optional[torch.Tensor]=None,
        lbl2data_data2ptr:Optional[torch.Tensor]=None,
        lbl2data_idx:Optional[torch.Tensor]=None,
        lbl2data_input_ids:Optional[torch.Tensor]=None,
        lbl2data_attention_mask:Optional[torch.Tensor]=None,
        plbl2data_data2ptr:Optional[torch.Tensor]=None,
        plbl2data_idx:Optional[torch.Tensor]=None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
        **kwargs
    ):  
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict
        
        if self.use_encoder_parallel: 
            encoder = XCDataParallel(module=self.encoder)
        else: encoder = self.encoder
        
        data_meta_kwargs = Parameters.from_feat_meta_aug_prefix('data', self.data_aug_meta_prefix, **kwargs)
        data_o = encoder(data_input_ids=data_input_ids, data_attention_mask=data_attention_mask, 
                         data_aug_meta_prefix=self.data_aug_meta_prefix, **data_meta_kwargs)

        if data_o.rep is not None: data_o.rep = F.normalize(self.transform(data_o.rep), dim=1)
        if data_o.fused_rep is not None: data_o.fused_rep = F.normalize(self.transform(data_o.fused_rep), dim=1)

        loss = None; lbl2data_o = EncoderOutput()
        if lbl2data_input_ids is not None:
            lbl2data_meta_kwargs = Parameters.from_feat_meta_aug_prefix('lbl2data', self.lbl2data_aug_meta_prefix, **kwargs)
            lbl2data_o = encoder(data_input_ids=lbl2data_input_ids, data_attention_mask=lbl2data_attention_mask, 
                                 data_aug_meta_prefix=self.lbl2data_aug_meta_prefix, **lbl2data_meta_kwargs)
            lbl2data_o.rep = F.normalize(lbl2data_o.rep + self.label_embeddings(self.label_remap[lbl2data_idx]), dim=1)

            if lbl2data_o.rep is not None: lbl2data_o.rep = F.normalize(self.transform(lbl2data_o.rep), dim=1)
            if lbl2data_o.fused_rep is not None: lbl2data_o.fused_rep = F.normalize(self.transform(lbl2data_o.fused_rep), dim=1)
            
            loss = self.compute_loss(data_o.fused_rep, lbl2data_o.rep,lbl2data_data2ptr,lbl2data_idx,
                                     plbl2data_data2ptr,plbl2data_idx)

            if self.use_query_loss:
                loss += self.compute_loss(data_o.rep, lbl2data_o.rep,lbl2data_data2ptr,lbl2data_idx,
                                          plbl2data_data2ptr,plbl2data_idx)

            if self.use_calib_loss:
                loss += self.calibration_loss(data_o.fused_rep, data_o.rep, lbl2data_o.rep,lbl2data_data2ptr,lbl2data_idx,
                                              plbl2data_data2ptr,plbl2data_idx)
            
            loss += self.compute_meta_loss(data_o.fused_rep, lbl2data_o.rep, **kwargs)
            
            if self.use_fusion_loss:
                loss += self.compute_fusion_loss(data_o.fused_rep, data_o.meta_repr, self.data_aug_meta_prefix, **kwargs)
                loss += self.compute_fusion_loss(lbl2data_o.rep, lbl2data_o.meta_repr, self.lbl2data_aug_meta_prefix, **kwargs)
            
            
        if not return_dict:
            o = (data_o.logits,data_o.rep,data_o.fused_rep,lbl2data_o.logits,lbl2data_o.rep,lbl2data_o.fused_rep)
            return ((loss,) + o) if loss is not None else o

        return XCModelOutput(
            loss=loss,
            
            data_repr=data_o.rep,
            data_fused_repr=data_o.fused_rep,
            
            lbl2data_repr=lbl2data_o.rep,
            lbl2data_fused_repr=lbl2data_o.fused_rep,
        )
        

### Examples

In [18]:
model = OAK009.from_pretrained('sentence-transformers/msmarco-distilbert-base-v4', batch_size=100, num_batch_labels=5000, 
                               margin=0.3, num_negatives=5, tau=0.1, apply_softmax=True,
                               
                               data_aug_meta_prefix='cat2data', lbl2data_aug_meta_prefix=None, 
                               data_pred_meta_prefix=None, lbl2data_pred_meta_prefix=None,
                               
                               num_metadata=block.train.dset.meta['cat_meta'].n_meta, resize_length=5000, 
                               n_labels=block.n_lbl, n_clusters=block.n_lbl//3, embed_dim=4096,
                               
                               calib_margin=0.3, calib_num_negatives=10, calib_tau=0.1, calib_apply_softmax=False, 
                               calib_loss_weight=0.1, use_calib_loss=False,

                               use_query_loss=True,

                               meta_loss_weight=0.0, 
                               
                               fusion_loss_weight=0.1, use_fusion_loss=False,
                               
                               use_encoder_parallel=False)

model.init_retrieval_head()
model.init_cross_head()
model.init_meta_embeddings()
model.init_label_embeddings()
model.init_transform()

Some weights of OAK009 were not initialized from the model checkpoint at sentence-transformers/msmarco-distilbert-base-v4 and are newly initialized: ['encoder.cross_head.k.bias', 'encoder.cross_head.k.weight', 'encoder.cross_head.o.bias', 'encoder.cross_head.o.weight', 'encoder.cross_head.q.bias', 'encoder.cross_head.q.weight', 'encoder.cross_head.v.bias', 'encoder.cross_head.v.weight', 'encoder.dr_fused_head.layer_norm.bias', 'encoder.dr_fused_head.layer_norm.weight', 'encoder.dr_fused_head.projector.bias', 'encoder.dr_fused_head.projector.weight', 'encoder.dr_fused_head.transform.bias', 'encoder.dr_fused_head.transform.weight', 'encoder.dr_head.layer_norm.bias', 'encoder.dr_head.layer_norm.weight', 'encoder.dr_head.projector.bias', 'encoder.dr_head.projector.weight', 'encoder.dr_head.transform.bias', 'encoder.dr_head.transform.weight', 'encoder.meta_embeddings.weight', 'encoder.meta_head.layer_norm.bias', 'encoder.meta_head.layer_norm.weight', 'encoder.meta_head.projector.bias', 'enc

In [19]:
model.encoder.set_pretrained_meta_embeddings(torch.zeros(656086, 768))
model.encoder.freeze_pretrained_meta_embeddings()

In [20]:
model = model.to('cuda')

In [21]:
b = prepare_batch(model, batch, m_args=[
    'pcat2data_idx', 'pcat2data_data2ptr', 'cat2data_idx', 'cat2data_input_ids', 'cat2data_attention_mask', 
    'cat2data_data2ptr',
])

In [22]:
def func():
    import pdb; pdb.set_trace()
    o = model(**b.to(model.device))
    

In [129]:
func()

> /tmp/ipykernel_18914/3721260802.py(3)func()
      1 def func():
      2     import pdb; pdb.set_trace()
----> 3     o = model(**b.to(model.device))
      4 



ipdb>  b model.forward


Breakpoint 2 at /tmp/ipykernel_18914/134643332.py:19


ipdb>  c


> /tmp/ipykernel_18914/134643332.py(35)forward()
     33         **kwargs
     34     ):  
---> 35         return_dict = return_dict if return_dict is not None else self.config.use_return_dict
     36 
     37         if self.use_encoder_parallel:



...

--Return--
None
> /tmp/ipykernel_18914/3721260802.py(3)func()
      1 def func():
      2     import pdb; pdb.set_trace()
----> 3     o = model(**b.to(model.device))
      4 



ipdb>  


--Call--
> /home/scai/phd/aiz218323/.local/lib/python3.9/site-packages/IPython/core/displayhook.py(258)__call__()
    256         sys.stdout.flush()
    257 
--> 258     def __call__(self, result=None):
    259         """Printing with history cache management.
    260 



ipdb>  c


    [... skipped 1 hidden frame]

    [... skipped 1 hidden frame]

    [... skipped 1 hidden frame]

    [... skipped 1 hidden frame]



In [23]:
o = model(**b.to(model.device))

/scratch/scai/phd/aiz218323/Projects/xcai/xcai/losses.py:22: UserWarning: Sparse CSR tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at ../aten/src/ATen/SparseCsrTensorImpl.cpp:53.)
  return torch.sparse_csr_tensor(data_ptr, data_idx, scores, device=data_ptr.device)


In [25]:
o.loss

tensor(0.0260, device='cuda:0', grad_fn=<AddBackward0>)

## `OAK010`

In [30]:
#| export
class Encoder010(Encoder003):

    def __init__(
        self, 
        config,
        n_clusters:int,
        n_metadata:int,
        **kwargs
    ):
        super().__init__(config, num_metadata=n_clusters, **kwargs)
        self.pretrained_meta_embeddings = nn.Embedding(n_metadata, config.dim)
        self.register_buffer("metadata_remap", torch.arange(n_metadata)%n_clusters, persistent=True)
        self.post_init()

    def set_metadata_remap(self, metadata_remap:torch.Tensor):
        if metadata_remap.shape[0] != self.metadata_remap.shape[0]:
            raise ValueError(f'Shape mismatch, `metadata_remap` should have {self.metadata_remap.shape[0]} elements.')
        self.metadata_remap = metadata_remap

    def fuse_meta_into_embeddings(self, data_repr:torch.Tensor, data_mask:torch.Tensor, meta_kwargs:Dict):
        meta_repr = {}
        
        data_fused_repr, data_mask = data_repr.clone().view(-1, 1, self.config.dim), data_mask.view(-1, 1)
        for m_key, m_args in meta_kwargs.items():
            idx = torch.where(m_args['data2ptr'] > 0)[0]
            meta_repr[m_key] = torch.empty(0, self.config.dim).to(data_repr)
            
            if len(idx):
                m_idx,m_repr_mask = self.resize(m_args['idx'], m_args['data2ptr'][idx])
                m_repr = F.normalize(self.meta_embeddings(self.metadata_remap[m_idx]) + self.pretrained_meta_embeddings(m_idx), dim=1)
                
                m_repr, m_repr_mask = m_repr.view(len(idx), -1, self.config.dim), m_repr_mask.bool().view(len(idx), -1)
                meta_repr[m_key] = m_repr[m_repr_mask]
                
                fused_repr = self.cross_head(data_fused_repr[idx], data_mask[idx], m_repr, m_repr_mask)[0]
                data_fused_repr[idx] += fused_repr
                
        return data_fused_repr.squeeze(), meta_repr
    

In [21]:
#| export
class OAK010(OAK000, DistilBertPreTrainedModel):
    use_generation,use_representation = False,True
    _tied_weights_keys = ["encoder.distilbert"]

    @delegates(OAK000.__init__)
    def __init__(
        self, 
        config,
        n_clusters:int,
        n_metadata:int,
        resize_length:Optional[int]=None,
        **kwargs
    ):
        super().__init__(config, **kwargs)
        self.encoder = Encoder010(config, n_clusters=n_clusters, n_metadata=n_metadata, resize_length=resize_length)
        self.post_init(); self.remap_post_init(); self.init_retrieval_head(); self.init_cross_head()

    def init_meta_embeddings(self):
        self.encoder.init_meta_embeddings()

    def remap_post_init(self):
        self.distilbert = self.encoder.distilbert
        

### Example

In [32]:
model = OAK010.from_pretrained('sentence-transformers/msmarco-distilbert-base-v4', batch_size=100, num_batch_labels=5000, 
                               margin=0.3, num_negatives=5, tau=0.1, apply_softmax=True,
                               
                               data_aug_meta_prefix='cat2data', lbl2data_aug_meta_prefix=None, 
                               data_pred_meta_prefix=None, lbl2data_pred_meta_prefix=None,
                               
                               n_metadata=block.train.dset.meta['cat_meta'].n_meta, n_clusters=1000, resize_length=5000,
                               
                               calib_margin=0.05, calib_num_negatives=10, calib_tau=0.1, calib_apply_softmax=False, 
                               calib_loss_weight=0.1, use_calib_loss=False,

                               use_query_loss=True,

                               meta_loss_weight=0.0, 
                               
                               fusion_loss_weight=0.0, use_fusion_loss=False,
                               
                               use_encoder_parallel=False)

model.init_retrieval_head()
model.init_cross_head()
model.init_meta_embeddings()

Some weights of OAK010 were not initialized from the model checkpoint at sentence-transformers/msmarco-distilbert-base-v4 and are newly initialized: ['encoder.cross_head.k.bias', 'encoder.cross_head.k.weight', 'encoder.cross_head.o.bias', 'encoder.cross_head.o.weight', 'encoder.cross_head.q.bias', 'encoder.cross_head.q.weight', 'encoder.cross_head.v.bias', 'encoder.cross_head.v.weight', 'encoder.dr_fused_head.layer_norm.bias', 'encoder.dr_fused_head.layer_norm.weight', 'encoder.dr_fused_head.projector.bias', 'encoder.dr_fused_head.projector.weight', 'encoder.dr_fused_head.transform.bias', 'encoder.dr_fused_head.transform.weight', 'encoder.dr_head.layer_norm.bias', 'encoder.dr_head.layer_norm.weight', 'encoder.dr_head.projector.bias', 'encoder.dr_head.projector.weight', 'encoder.dr_head.transform.bias', 'encoder.dr_head.transform.weight', 'encoder.meta_embeddings.weight', 'encoder.meta_head.layer_norm.bias', 'encoder.meta_head.layer_norm.weight', 'encoder.meta_head.projector.bias', 'enc

In [33]:
model.encoder.set_pretrained_meta_embeddings(torch.zeros(656086, 768))
model.encoder.freeze_pretrained_meta_embeddings()

remap = torch.arange(block.train.dset.meta['cat_meta'].n_meta)%1000
model.encoder.set_metadata_remap(remap)

In [34]:
model = model.to('cuda')

In [ ]:
b = prepare_batch(model, batch, m_args=[
    'plnk2data_idx', 'plnk2data_data2ptr', 'lnk2data_idx', 'lnk2data_input_ids', 'lnk2data_attention_mask', 
    'lnk2data_data2ptr',
])

In [35]:
b = prepare_batch(model, batch, m_args=[
    'pcat2data_idx', 'pcat2data_data2ptr', 'cat2data_idx', 'cat2data_input_ids', 'cat2data_attention_mask', 
    'cat2data_data2ptr',
])

In [36]:
o = model(**b.to(model.device))

In [37]:
o.loss

tensor(0.0260, device='cuda:0', grad_fn=<AddBackward0>)

## `OAK011`

In [26]:
#| export
class Encoder011(Encoder003):
    
    def __init__(
        self, 
        config, 
        n_clusters:int,
        n_metadata:int,
        **kwargs
    ):
        super().__init__(config, num_metadata=n_clusters, **kwargs)
        self.pretrained_meta_embeddings = nn.Embedding(n_clusters, config.dim)
        self.register_buffer("metadata_remap", torch.arange(n_metadata)%n_clusters, persistent=True)
        self.post_init()

    def set_metadata_remap(self, metadata_remap:torch.Tensor):
        if metadata_remap.shape[0] != self.metadata_remap.shape[0]:
            raise ValueError(f'Shape mismatch, `metadata_remap` should have {self.metadata_remap.shape[0]} elements.')
        self.metadata_remap = metadata_remap

    def fuse_meta_into_embeddings(self, data_repr:torch.Tensor, data_mask:torch.Tensor, meta_kwargs:Dict):
        meta_repr = {}
        
        data_fused_repr, data_mask = data_repr.clone().view(-1, 1, self.config.dim), data_mask.view(-1, 1)
        for m_key, m_args in meta_kwargs.items():
            idx = torch.where(m_args['data2ptr'] > 0)[0]
            meta_repr[m_key] = torch.empty(0, self.config.dim).to(data_repr)
            
            if len(idx):
                m_idx,m_repr_mask = self.resize(m_args['idx'], m_args['data2ptr'][idx])
                m_repr = F.normalize(self.meta_embeddings(self.metadata_remap[m_idx]) + self.pretrained_meta_embeddings(self.metadata_remap[m_idx]), dim=1)
                
                m_repr, m_repr_mask = m_repr.view(len(idx), -1, self.config.dim), m_repr_mask.bool().view(len(idx), -1)
                meta_repr[m_key] = m_repr[m_repr_mask]
                
                fused_repr = self.cross_head(data_fused_repr[idx], data_mask[idx], m_repr, m_repr_mask)[0]
                data_fused_repr[idx] += fused_repr
                
        return data_fused_repr.squeeze(), meta_repr

    

In [27]:
#| export
class OAK011(OAK000, DistilBertPreTrainedModel):
    use_generation,use_representation = False,True
    _tied_weights_keys = ["encoder.distilbert"]

    @delegates(OAK000.__init__)
    def __init__(
        self, 
        config,
        n_clusters:int,
        n_metadata:int,
        resize_length:Optional[int]=None,
        **kwargs
    ):
        super().__init__(config, **kwargs)
        self.encoder = Encoder011(config, n_clusters=n_clusters, n_metadata=n_metadata, resize_length=resize_length)
        self.post_init(); self.remap_post_init(); self.init_retrieval_head(); self.init_cross_head()

    def init_meta_embeddings(self):
        self.encoder.init_meta_embeddings()

    def remap_post_init(self):
        self.distilbert = self.encoder.distilbert

### Example

In [28]:
model = OAK011.from_pretrained('sentence-transformers/msmarco-distilbert-base-v4', batch_size=100, num_batch_labels=5000, 
                               margin=0.3, num_negatives=5, tau=0.1, apply_softmax=True,
                               
                               data_aug_meta_prefix='cat2data', lbl2data_aug_meta_prefix=None, 
                               data_pred_meta_prefix=None, lbl2data_pred_meta_prefix=None,
                               
                               n_metadata=block.train.dset.meta['cat_meta'].n_meta, n_clusters=1000, resize_length=5000,
                               
                               calib_margin=0.05, calib_num_negatives=10, calib_tau=0.1, calib_apply_softmax=False, 
                               calib_loss_weight=0.1, use_calib_loss=False,

                               use_query_loss=True,

                               meta_loss_weight=0.0, 
                               
                               fusion_loss_weight=0.0, use_fusion_loss=False,
                               
                               use_encoder_parallel=False)

model.init_retrieval_head()
model.init_cross_head()
model.init_meta_embeddings()

Some weights of OAK011 were not initialized from the model checkpoint at sentence-transformers/msmarco-distilbert-base-v4 and are newly initialized: ['encoder.cross_head.k.bias', 'encoder.cross_head.k.weight', 'encoder.cross_head.o.bias', 'encoder.cross_head.o.weight', 'encoder.cross_head.q.bias', 'encoder.cross_head.q.weight', 'encoder.cross_head.v.bias', 'encoder.cross_head.v.weight', 'encoder.dr_fused_head.layer_norm.bias', 'encoder.dr_fused_head.layer_norm.weight', 'encoder.dr_fused_head.projector.bias', 'encoder.dr_fused_head.projector.weight', 'encoder.dr_fused_head.transform.bias', 'encoder.dr_fused_head.transform.weight', 'encoder.dr_head.layer_norm.bias', 'encoder.dr_head.layer_norm.weight', 'encoder.dr_head.projector.bias', 'encoder.dr_head.projector.weight', 'encoder.dr_head.transform.bias', 'encoder.dr_head.transform.weight', 'encoder.meta_embeddings.weight', 'encoder.meta_head.layer_norm.bias', 'encoder.meta_head.layer_norm.weight', 'encoder.meta_head.projector.bias', 'enc

In [29]:
model.encoder.set_pretrained_meta_embeddings(torch.zeros(656086, 768))
model.encoder.freeze_pretrained_meta_embeddings()

remap = torch.arange(block.train.dset.meta['cat_meta'].n_meta)%1000
model.encoder.set_metadata_remap(remap)

In [30]:
b = prepare_batch(model, batch, m_args=[
    'pcat2data_idx', 'pcat2data_data2ptr', 'cat2data_idx', 'cat2data_input_ids', 'cat2data_attention_mask', 
    'cat2data_data2ptr',
])

In [31]:
o = model(**b.to(model.device))

/scratch/scai/phd/aiz218323/Projects/xcai/xcai/losses.py:22: UserWarning: Sparse CSR tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at ../aten/src/ATen/SparseCsrTensorImpl.cpp:53.)
  return torch.sparse_csr_tensor(data_ptr, data_idx, scores, device=data_ptr.device)


In [32]:
o.loss

tensor(0.0252, grad_fn=<AddBackward0>)

## `OAK012`

In [37]:
#| export
class Encoder012(Encoder):

    def __init__(self, config, **kwargs):
        super().__init__(config, **kwargs)
        self.meta_transform = RepresentationHead(config)
        self.post_init()

    def fuse_meta_into_embeddings(self, data_repr:torch.Tensor, data_mask:torch.Tensor, meta_kwargs:Dict):
        meta_repr = {}
        
        data_fused_repr, data_mask = data_repr.clone().view(-1, 1, self.config.dim), data_mask.view(-1, 1)
        for m_key, m_args in meta_kwargs.items():
            idx = torch.where(m_args['data2ptr'] > 0)[0]
            meta_repr[m_key] = torch.empty(0, self.config.dim).to(data_repr)
            
            if len(idx):
                m_idx,m_repr_mask = self.resize(m_args['idx'], m_args['data2ptr'][idx])
                m_repr = F.normalize(self.meta_transform(self.meta_embeddings(m_idx)), dim=1)
                
                m_repr, m_repr_mask = m_repr.view(len(idx), -1, self.config.dim), m_repr_mask.bool().view(len(idx), -1)
                meta_repr[m_key] = m_repr[m_repr_mask]
                
                fused_repr = self.cross_head(data_fused_repr[idx], data_mask[idx], m_repr, m_repr_mask)[0]
                data_fused_repr[idx] += fused_repr
                
        return data_fused_repr.squeeze(), meta_repr
    

In [38]:
#| export
class OAK012(OAK000, DistilBertPreTrainedModel):
    use_generation,use_representation = False,True
    _tied_weights_keys = ["encoder.distilbert"]

    @delegates(OAK000.__init__)
    def __init__(
        self, 
        config,
        num_metadata:int,
        resize_length:Optional[int]=None,
        **kwargs
    ):
        super().__init__(config, **kwargs)
        self.encoder = Encoder012(config, num_metadata=num_metadata, resize_length=resize_length)
        self.post_init(); self.remap_post_init(); self.init_retrieval_head(); self.init_cross_head()
        
    def remap_post_init(self):
        self.distilbert = self.encoder.distilbert
        

### Example

In [39]:
model = OAK012.from_pretrained('sentence-transformers/msmarco-distilbert-base-v4', batch_size=100, num_batch_labels=5000, 
                               margin=0.3, num_negatives=5, tau=0.1, apply_softmax=True,
                               
                               data_aug_meta_prefix='cat2data', lbl2data_aug_meta_prefix=None, 
                               data_pred_meta_prefix=None, lbl2data_pred_meta_prefix=None,
                               
                               num_metadata=block.train.dset.meta['cat_meta'].n_meta, resize_length=5000,
                               
                               calib_margin=0.3, calib_num_negatives=10, calib_tau=0.1, calib_apply_softmax=False, 
                               calib_loss_weight=0.1, use_calib_loss=False,

                               use_query_loss=True,

                               meta_loss_weight=0.3, 
                               
                               fusion_loss_weight=0.1, use_fusion_loss=False,
                               
                               use_encoder_parallel=False)

model.init_retrieval_head()
model.init_cross_head()

Some weights of OAK012 were not initialized from the model checkpoint at sentence-transformers/msmarco-distilbert-base-v4 and are newly initialized: ['encoder.cross_head.k.bias', 'encoder.cross_head.k.weight', 'encoder.cross_head.o.bias', 'encoder.cross_head.o.weight', 'encoder.cross_head.q.bias', 'encoder.cross_head.q.weight', 'encoder.cross_head.v.bias', 'encoder.cross_head.v.weight', 'encoder.dr_fused_head.layer_norm.bias', 'encoder.dr_fused_head.layer_norm.weight', 'encoder.dr_fused_head.projector.bias', 'encoder.dr_fused_head.projector.weight', 'encoder.dr_fused_head.transform.bias', 'encoder.dr_fused_head.transform.weight', 'encoder.dr_head.layer_norm.bias', 'encoder.dr_head.layer_norm.weight', 'encoder.dr_head.projector.bias', 'encoder.dr_head.projector.weight', 'encoder.dr_head.transform.bias', 'encoder.dr_head.transform.weight', 'encoder.meta_embeddings.weight', 'encoder.meta_head.layer_norm.bias', 'encoder.meta_head.layer_norm.weight', 'encoder.meta_head.projector.bias', 'enc

In [41]:
model.encoder.meta_transform.post_init()

In [43]:
model.encoder.set_meta_embeddings(torch.zeros(656086, 768))

In [44]:
model = model.to('cuda')

In [45]:
b = prepare_batch(model, batch, m_args=[
    'pcat2data_idx', 'pcat2data_data2ptr', 'cat2data_idx', 'cat2data_input_ids', 'cat2data_attention_mask', 
    'cat2data_data2ptr',
])

In [46]:
b.keys()

dict_keys(['plbl2data_idx', 'plbl2data_data2ptr', 'lbl2data_idx', 'lbl2data_input_ids', 'lbl2data_attention_mask', 'lbl2data_data2ptr', 'cat2data_idx', 'cat2data_input_ids', 'cat2data_attention_mask', 'cat2data_data2ptr', 'data_input_ids', 'data_attention_mask'])

In [47]:
o = model(**b.to(model.device))

In [48]:
o.loss

tensor(0.0260, device='cuda:0', grad_fn=<AddBackward0>)

## `OAK013`

In [55]:
#| export
class Encoder013(Encoder003):

    def __init__(
        self, 
        config,
        **kwargs
    ):
        super().__init__(config, **kwargs)
        self.meta_distilbert = DistilBertModel(config)
        self.post_init()

    def meta_encode(self, input_ids:torch.Tensor, attention_mask:torch.Tensor, **kwargs):
        o = self.meta_distilbert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            **kwargs
        )
        return Pooling.mean_pooling(o[0], attention_mask)

    def init_meta_encoder(self):
        sd_meta, sd_dr = self.meta_distilbert.state_dict(), self.distilbert.state_dict()
        for k in sd_dr:
            assert sd_meta[k].shape == sd_dr[k].shape
            with torch.no_grad():
                sd_meta[k].copy_(sd_dr[k])

    def fuse_meta_into_embeddings(self, data_repr:torch.Tensor, data_mask:torch.Tensor, meta_kwargs:Dict):
        meta_repr = {}
        
        data_fused_repr, data_mask = data_repr.clone().view(-1, 1, self.config.dim), data_mask.view(-1, 1)
        for m_key, m_args in meta_kwargs.items():
            n_meta, bsz = m_args['data2ptr'].max(), len(m_args['data2ptr'])
            assert torch.all(m_args['data2ptr'] == n_meta), f'All datapoints should have same number of metadata.'
            
            m_input_ids, m_attention_mask = m_args['input_ids'], m_args['attention_mask']
            m_embed = self.meta_encode(input_ids=m_input_ids, attention_mask=m_attention_mask)
            
            m_idx = m_args['idx']
            m_repr = F.normalize(self.meta_embeddings(m_idx) + m_embed + self.pretrained_meta_embeddings(m_idx), dim=1)
            
            m_repr, m_repr_mask = m_repr.view(bsz, -1, self.config.dim), torch.ones((m_repr.shape[0],), device=m_repr.device, dtype=torch.bool).view(bsz, -1)
            meta_repr[m_key] = m_repr[m_repr_mask]
            
            fused_repr = self.cross_head(data_fused_repr, data_mask, m_repr, m_repr_mask)[0]
            data_fused_repr += fused_repr
                
        return data_fused_repr.squeeze(), meta_repr
    

In [56]:
#| export
class OAK013(OAK008, DistilBertPreTrainedModel):
    use_generation,use_representation = False,True
    _tied_weights_keys = ["encoder.distilbert"]
    _keys_to_ignore_on_load_missing = ["encoder.meta_distilbert"]

    @delegates(OAK008.__init__)
    def __init__(
        self, 
        config,
        num_metadata:int,
        resize_length:Optional[int]=None,
        **kwargs
    ):
        super().__init__(config, num_metadata=num_metadata, resize_length=resize_length, **kwargs)
        self.encoder = Encoder013(config, num_metadata=num_metadata, resize_length=resize_length)
        self.post_init(); self.remap_post_init(); self.init_retrieval_head(); self.init_cross_head()

    def remap_post_init(self):
        self.distilbert = self.encoder.distilbert

    def init_meta_encoder(self):
        self.encoder.init_meta_encoder()
        

### Example

In [57]:
model = OAK013.from_pretrained('sentence-transformers/msmarco-distilbert-base-v4', batch_size=100, num_batch_labels=5000, 
                               margin=0.3, num_negatives=5, tau=0.1, apply_softmax=True,
                               
                               data_aug_meta_prefix='lnk2data', lbl2data_aug_meta_prefix=None, 
                               data_pred_meta_prefix=None, lbl2data_pred_meta_prefix=None,
                               
                               num_metadata=block.train.dset.meta['lnk_meta'].n_meta, resize_length=5000, 
                               n_labels=block.n_lbl, n_clusters=block.n_lbl//3,
                               
                               calib_margin=0.3, calib_num_negatives=10, calib_tau=0.1, calib_apply_softmax=False, 
                               calib_loss_weight=0.1, use_calib_loss=False,

                               use_query_loss=True,

                               meta_loss_weight=0.0, 
                               
                               fusion_loss_weight=0.1, use_fusion_loss=False,
                               
                               use_encoder_parallel=False)

model.init_retrieval_head()
model.init_cross_head()
model.init_meta_embeddings()
model.init_label_embeddings()

Some weights of OAK013 were not initialized from the model checkpoint at sentence-transformers/msmarco-distilbert-base-v4 and are newly initialized: ['encoder.cross_head.k.bias', 'encoder.cross_head.k.weight', 'encoder.cross_head.o.bias', 'encoder.cross_head.o.weight', 'encoder.cross_head.q.bias', 'encoder.cross_head.q.weight', 'encoder.cross_head.v.bias', 'encoder.cross_head.v.weight', 'encoder.dr_fused_head.layer_norm.bias', 'encoder.dr_fused_head.layer_norm.weight', 'encoder.dr_fused_head.projector.bias', 'encoder.dr_fused_head.projector.weight', 'encoder.dr_fused_head.transform.bias', 'encoder.dr_fused_head.transform.weight', 'encoder.dr_head.layer_norm.bias', 'encoder.dr_head.layer_norm.weight', 'encoder.dr_head.projector.bias', 'encoder.dr_head.projector.weight', 'encoder.dr_head.transform.bias', 'encoder.dr_head.transform.weight', 'encoder.meta_embeddings.weight', 'encoder.meta_head.layer_norm.bias', 'encoder.meta_head.layer_norm.weight', 'encoder.meta_head.projector.bias', 'enc

In [58]:
model.encoder.set_pretrained_meta_embeddings(torch.zeros(656086, 768))
model.encoder.freeze_pretrained_meta_embeddings()

model.init_meta_encoder()

In [23]:
model = model.to('cuda')

In [24]:
b = prepare_batch(model, batch, m_args=[
    'plnk2data_idx', 'plnk2data_data2ptr', 'lnk2data_idx', 'lnk2data_input_ids', 'lnk2data_attention_mask', 
    'lnk2data_data2ptr',
])

In [25]:
o = model(**b.to(model.device))

/scratch/scai/phd/aiz218323/Projects/xcai/xcai/losses.py:22: UserWarning: Sparse CSR tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at ../aten/src/ATen/SparseCsrTensorImpl.cpp:53.)
  return torch.sparse_csr_tensor(data_ptr, data_idx, scores, device=data_ptr.device)


In [99]:
o.loss

tensor(0.0149, device='cuda:0', grad_fn=<AddBackward0>)

In [100]:
def func():
    import pdb; pdb.set_trace()
    return model(**b.to(model.device))

In [101]:
o = func()

> /tmp/ipykernel_26512/1795951242.py(3)func()
      1 def func():
      2     import pdb; pdb.set_trace()
----> 3     return model(**b.to(model.device))



ipdb>  b model.forward


Breakpoint 1 at /tmp/ipykernel_26512/2049060262.py:45


ipdb>  b model.encoder.forward


Breakpoint 2 at /tmp/ipykernel_26512/4122862164.py:106


...

> /tmp/ipykernel_26512/4122862164.py(117)forward()
    115         data_o = self.encode(data_input_ids, data_attention_mask)
    116 
--> 117         if data_type is not None and data_type == "meta":
    118             data_repr = self.meta_unnormalized(data_o[0], data_attention_mask) if data_unnormalized else self.meta(data_o[0], data_attention_mask)
    119         else:



ipdb>  


> /tmp/ipykernel_26512/4122862164.py(120)forward()
    118             data_repr = self.meta_unnormalized(data_o[0], data_attention_mask) if data_unnormalized else self.meta(data_o[0], data_attention_mask)
    119         else:
--> 120             data_repr = self.dr(data_o[0], data_attention_mask)
    121 
    122         data_fused_repr = meta_repr = None



ipdb>  c


    [... skipped 1 hidden frame]

    [... skipped 1 hidden frame]

    [... skipped 1 hidden frame]

    [... skipped 1 hidden frame]



In [102]:
o.loss

tensor(0.0149, grad_fn=<AddBackward0>)

## `OAK014`

In [118]:
#| export
class Encoder014(Encoder013):

    def __init__(
        self,
        config,
        n_meta_clusters:int,
        n_metadata:int,
        **kwargs
    ):
        super().__init__(config, num_metadata=n_meta_clusters, **kwargs)
        self.pretrained_meta_embeddings = nn.Embedding(n_meta_clusters, config.dim)
        self.register_buffer("metadata_remap", torch.arange(n_metadata)%n_meta_clusters, persistent=True)
        self.post_init()

    def set_metadata_remap(self, metadata_remap:torch.Tensor):
        if metadata_remap.shape[0] != self.metadata_remap.shape[0]:
            raise ValueError(f'Shape mismatch, `metadata_remap` should have {self.metadata_remap.shape[0]} elements.')
        with torch.no_grad():
            self.metadata_remap.copy_(metadata_remap)

    def fuse_meta_into_embeddings(self, data_repr:torch.Tensor, data_mask:torch.Tensor, meta_kwargs:Dict):
        meta_repr = {}
        
        data_fused_repr, data_mask = data_repr.clone().view(-1, 1, self.config.dim), data_mask.view(-1, 1)
        for m_key, m_args in meta_kwargs.items():
            n_meta, bsz = m_args['data2ptr'].max(), len(m_args['data2ptr'])
            assert torch.all(m_args['data2ptr'] == n_meta), f'All datapoints should have same number of metadata.'
            
            m_input_ids, m_attention_mask = m_args['input_ids'], m_args['attention_mask']
            m_embed = self.meta_encode(input_ids=m_input_ids, attention_mask=m_attention_mask)
            
            m_idx = m_args['idx']
            m_repr = F.normalize(self.meta_embeddings(self.metadata_remap[m_idx]) + m_embed + self.pretrained_meta_embeddings(self.metadata_remap[m_idx]), dim=1)
            
            m_repr, m_repr_mask = m_repr.view(bsz, -1, self.config.dim), torch.ones((m_repr.shape[0],), device=m_repr.device, dtype=torch.bool).view(bsz, -1)
            meta_repr[m_key] = m_repr[m_repr_mask]
            
            fused_repr = self.cross_head(data_fused_repr, data_mask, m_repr, m_repr_mask)[0]
            data_fused_repr += fused_repr
                
        return data_fused_repr.squeeze(), meta_repr
        

In [122]:
#| export
class OAK014(OAK013, DistilBertPreTrainedModel):
    use_generation,use_representation = False,True
    _tied_weights_keys = ["encoder.distilbert"]
    _keys_to_ignore_on_load_missing = ["encoder.meta_distilbert"]

    @delegates(OAK013.__init__)
    def __init__(
        self, 
        config,
        n_metadata:int,
        n_meta_clusters:int,
        resize_length:Optional[int]=None,
        **kwargs
    ):
        super().__init__(config, num_metadata=n_metadata, resize_length=resize_length, **kwargs)
        self.encoder = Encoder014(config, n_metadata=n_metadata, n_meta_clusters=n_meta_clusters, resize_length=resize_length)
        self.post_init(); self.remap_post_init(); self.init_retrieval_head(); self.init_cross_head()
        
        

### Example

In [123]:
model = OAK014.from_pretrained('sentence-transformers/msmarco-distilbert-base-v4', batch_size=100, num_batch_labels=5000, 
                               margin=0.3, num_negatives=5, tau=0.1, apply_softmax=True,
                               
                               data_aug_meta_prefix='lnk2data', lbl2data_aug_meta_prefix=None, 
                               data_pred_meta_prefix=None, lbl2data_pred_meta_prefix=None,
                               
                               n_metadata=block.train.dset.meta['lnk_meta'].n_meta, resize_length=5000, n_meta_clusters=block.train.dset.meta['lnk_meta'].n_meta//3,
                               n_labels=block.n_lbl, n_clusters=block.n_lbl//3, 
                               
                               calib_margin=0.3, calib_num_negatives=10, calib_tau=0.1, calib_apply_softmax=False, 
                               calib_loss_weight=0.1, use_calib_loss=False,

                               use_query_loss=True,

                               meta_loss_weight=0.0, 
                               
                               fusion_loss_weight=0.1, use_fusion_loss=False,
                               
                               use_encoder_parallel=False)

model.init_retrieval_head()
model.init_cross_head()
model.init_meta_embeddings()
model.init_label_embeddings()

Some weights of OAK014 were not initialized from the model checkpoint at sentence-transformers/msmarco-distilbert-base-v4 and are newly initialized: ['encoder.cross_head.k.bias', 'encoder.cross_head.k.weight', 'encoder.cross_head.o.bias', 'encoder.cross_head.o.weight', 'encoder.cross_head.q.bias', 'encoder.cross_head.q.weight', 'encoder.cross_head.v.bias', 'encoder.cross_head.v.weight', 'encoder.dr_fused_head.layer_norm.bias', 'encoder.dr_fused_head.layer_norm.weight', 'encoder.dr_fused_head.projector.bias', 'encoder.dr_fused_head.projector.weight', 'encoder.dr_fused_head.transform.bias', 'encoder.dr_fused_head.transform.weight', 'encoder.dr_head.layer_norm.bias', 'encoder.dr_head.layer_norm.weight', 'encoder.dr_head.projector.bias', 'encoder.dr_head.projector.weight', 'encoder.dr_head.transform.bias', 'encoder.dr_head.transform.weight', 'encoder.meta_embeddings.weight', 'encoder.meta_head.layer_norm.bias', 'encoder.meta_head.layer_norm.weight', 'encoder.meta_head.projector.bias', 'enc

In [124]:
model.encoder.set_pretrained_meta_embeddings(torch.zeros(656086, 768))
model.encoder.freeze_pretrained_meta_embeddings()

model.init_meta_encoder()

remap = torch.arange(block.train.dset.meta['lnk_meta'].n_meta)%1000
model.encoder.set_metadata_remap(remap)

In [51]:
model = model.to('cuda')

In [52]:
b = prepare_batch(model, batch, m_args=[
    'plnk2data_idx', 'plnk2data_data2ptr', 'lnk2data_idx', 'lnk2data_input_ids', 'lnk2data_attention_mask', 
    'lnk2data_data2ptr',
])

In [53]:
o = model(**b.to(model.device))

In [54]:
o.loss

tensor(0.0149, device='cuda:0', grad_fn=<AddBackward0>)

In [116]:
def func():
    import pdb; pdb.set_trace()
    return model(**b.to(model.device))

In [ ]:
o = func()

> /tmp/ipykernel_26512/1795951242.py(3)func()
      1 def func():
      2     import pdb; pdb.set_trace()
----> 3     return model(**b.to(model.device))



ipdb>  b model.forward


Breakpoint 3 at /tmp/ipykernel_26512/2049060262.py:45


ipdb>  b model.encoder.forward


Breakpoint 4 at /tmp/ipykernel_26512/4122862164.py:106


...

> /tmp/ipykernel_26512/1735825316.py(30)fuse_meta_into_embeddings()
     28             assert torch.all(m_args['data2ptr'] == n_meta), f'All datapoints should have same number of metadata.'
     29 
---> 30             m_input_ids, m_attention_mask = m_args['input_ids'], m_args['attention_mask']
     31             m_embed = self.meta_encode(input_ids=m_input_ids, attention_mask=m_attention_mask)
     32 



ipdb>  


> /tmp/ipykernel_26512/1735825316.py(31)fuse_meta_into_embeddings()
     29 
     30             m_input_ids, m_attention_mask = m_args['input_ids'], m_args['attention_mask']
---> 31             m_embed = self.meta_encode(input_ids=m_input_ids, attention_mask=m_attention_mask)
     32 
     33             m_idx = m_args['idx']



ipdb>  


> /tmp/ipykernel_26512/1735825316.py(33)fuse_meta_into_embeddings()
     31             m_embed = self.meta_encode(input_ids=m_input_ids, attention_mask=m_attention_mask)
     32 
---> 33             m_idx = m_args['idx']
     34             m_repr = F.normalize(self.meta_embeddings(self.metadata_remap[m_idx]) + m_embed + self.pretrained_meta_embeddings(self.metadata_remap[m_idx]), dim=1)
     35 



## `OAK015`

In [24]:
#| export
class OAK015(OAK003):

    def __init__(
        self, 
        config,
        neg2data_aug_meta_prefix:Optional[str]=None,
        **kwargs,
    ):
        super().__init__(config, **kwargs)
        store_attr('neg2data_aug_meta_prefix')
        self.rep_loss_fn = MultiTripletWithNegatives(margin=kwargs['margin'], n_negatives=kwargs['num_negatives'], 
                                                     tau=kwargs['tau'], apply_softmax=kwargs['apply_softmax'], 
                                                     reduce='mean')
        
    def compute_loss(self, data_repr, lbl2data_repr, lbl2data_data2ptr, lbl2data_idx, neg2data_repr, 
                     neg2data_data2ptr, neg2data_idx, plbl2data_data2ptr, plbl2data_idx, **kwargs):
        return self.rep_loss_fn(data_repr, pos_targ=lbl2data_repr, n_pos=lbl2data_data2ptr, pos_idx=lbl2data_idx, 
                                neg_targ=neg2data_repr, n_neg=neg2data_data2ptr, neg_idx=neg2data_idx, 
                                n_ppos=plbl2data_data2ptr, ppos_idx=plbl2data_idx, **kwargs)
        
    def forward(
        self,
        data_input_ids:Optional[torch.Tensor]=None,
        data_attention_mask:Optional[torch.Tensor]=None,
        
        lbl2data_data2ptr:Optional[torch.Tensor]=None,
        lbl2data_idx:Optional[torch.Tensor]=None,
        lbl2data_input_ids:Optional[torch.Tensor]=None,
        lbl2data_attention_mask:Optional[torch.Tensor]=None,

        neg2data_input_ids:Optional[torch.Tensor]=None,
        neg2data_attention_mask:Optional[torch.Tensor]=None,
        neg2data_data2ptr:Optional[torch.Tensor]=None,
        neg2data_idx:Optional[torch.Tensor]=None,
        
        plbl2data_data2ptr:Optional[torch.Tensor]=None,
        plbl2data_idx:Optional[torch.Tensor]=None,
        
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
        **kwargs
    ):  
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict
        
        if self.use_encoder_parallel: 
            encoder = XCDataParallel(module=self.encoder)
        else: encoder = self.encoder
        
        data_meta_kwargs = Parameters.from_feat_meta_aug_prefix('data', self.data_aug_meta_prefix, **kwargs)
        data_o = encoder(data_input_ids=data_input_ids, data_attention_mask=data_attention_mask, 
                         data_aug_meta_prefix=self.data_aug_meta_prefix, **data_meta_kwargs)
        
        loss = None; lbl2data_o = EncoderOutput()
        if (
            lbl2data_input_ids is not None and 
            neg2data_input_ids is not None
        ):
            lbl2data_meta_kwargs = Parameters.from_feat_meta_aug_prefix('lbl2data', self.lbl2data_aug_meta_prefix, **kwargs)
            lbl2data_o = encoder(data_input_ids=lbl2data_input_ids, data_attention_mask=lbl2data_attention_mask, 
                                 data_aug_meta_prefix=self.lbl2data_aug_meta_prefix, **lbl2data_meta_kwargs)

            neg2data_meta_kwargs = Parameters.from_feat_meta_aug_prefix('neg2data', self.neg2data_aug_meta_prefix, **kwargs)
            neg2data_o = encoder(data_input_ids=neg2data_input_ids, data_attention_mask=neg2data_attention_mask, 
                                 data_aug_meta_prefix=self.neg2data_aug_meta_prefix, **neg2data_meta_kwargs)
            
            loss = self.compute_loss(data_o.fused_rep, lbl2data_o.rep, lbl2data_data2ptr, lbl2data_idx, neg2data_o.rep, 
                                     neg2data_data2ptr, neg2data_idx, plbl2data_data2ptr, plbl2data_idx, **kwargs)
            
            if self.use_query_loss:
                loss += self.compute_loss(data_o.rep, lbl2data_o.rep, lbl2data_data2ptr, lbl2data_idx, neg2data_o.rep, 
                                          neg2data_data2ptr, neg2data_idx, plbl2data_data2ptr, plbl2data_idx, **kwargs)
                
            if self.use_calib_loss:
                loss += self.calibration_loss(data_o.fused_rep, data_o.rep, lbl2data_o.rep, lbl2data_data2ptr, lbl2data_idx,
                                              plbl2data_data2ptr,plbl2data_idx)
                loss += self.calibration_loss(data_o.fused_rep, data_o.rep, neg2data_o.rep, neg2data_data2ptr, neg2data_idx,
                                              plbl2data_data2ptr,plbl2data_idx)
                
        if not return_dict:
            o = (data_o.logits,data_o.rep,data_o.fused_rep,lbl2data_o.logits,lbl2data_o.rep,lbl2data_o.fused_rep)
            return ((loss,) + o) if loss is not None else o
        
        
        return XCModelOutput(
            loss=loss,
            
            data_repr=data_o.rep,
            data_fused_repr=data_o.fused_rep,
            
            lbl2data_repr=lbl2data_o.rep,
            lbl2data_fused_repr=lbl2data_o.fused_rep,
        )
            

### Example

In [79]:
model = OAK015.from_pretrained('sentence-transformers/msmarco-distilbert-base-v4', margin=0.3, num_negatives=5, 
                               tau=0.1, apply_softmax=True,
                               
                               data_aug_meta_prefix='cat2data', lbl2data_aug_meta_prefix=None,
                               neg2data_aug_meta_prefix=None,
                               
                               num_metadata=block.train.dset.meta['cat_meta'].n_meta, resize_length=5000,
                               
                               calib_margin=0.3, calib_num_negatives=10, calib_tau=0.1, calib_apply_softmax=False, 
                               calib_loss_weight=0.1, use_calib_loss=False,

                               use_query_loss=True,

                               meta_loss_weight=0.3, 
                               
                               fusion_loss_weight=0.1, use_fusion_loss=True,
                               
                               use_encoder_parallel=False, normalize=True)

model.init_retrieval_head()
model.init_cross_head()
model.init_meta_embeddings()

Some weights of OAK015 were not initialized from the model checkpoint at sentence-transformers/msmarco-distilbert-base-v4 and are newly initialized: ['encoder.cross_head.k.bias', 'encoder.cross_head.k.weight', 'encoder.cross_head.o.bias', 'encoder.cross_head.o.weight', 'encoder.cross_head.q.bias', 'encoder.cross_head.q.weight', 'encoder.cross_head.v.bias', 'encoder.cross_head.v.weight', 'encoder.dr_fused_head.layer_norm.bias', 'encoder.dr_fused_head.layer_norm.weight', 'encoder.dr_fused_head.projector.bias', 'encoder.dr_fused_head.projector.weight', 'encoder.dr_fused_head.transform.bias', 'encoder.dr_fused_head.transform.weight', 'encoder.dr_head.layer_norm.bias', 'encoder.dr_head.layer_norm.weight', 'encoder.dr_head.projector.bias', 'encoder.dr_head.projector.weight', 'encoder.dr_head.transform.bias', 'encoder.dr_head.transform.weight', 'encoder.meta_embeddings.weight', 'encoder.meta_head.layer_norm.bias', 'encoder.meta_head.layer_norm.weight', 'encoder.meta_head.projector.bias', 'enc

In [96]:
model.encoder.set_pretrained_meta_embeddings(torch.zeros(656086, 768))
model.encoder.freeze_pretrained_meta_embeddings()

In [ ]:
model = model.to('cuda')

In [102]:
b = prepare_batch(model, batch, m_args=[
    'pcat2data_idx', 'pcat2data_data2ptr', 'cat2data_idx', 'cat2data_input_ids', 'cat2data_attention_mask', 
    'cat2data_data2ptr',
])

In [108]:
o = model(**b.to(model.device))

In [105]:
o.loss

tensor(0.1556, grad_fn=<AddBackward0>)

In [106]:
def func():
    import pdb; pdb.set_trace()
    o = model(**b.to(model.device))
    

In [110]:
func()

> /var/folders/x0/r_2wlyls39s3_q5g99w33dn80000gn/T/ipykernel_22918/3721260802.py(2)func()
      1 def func():
----> 2     import pdb; pdb.set_trace()
      3     o = model(**b.to(model.device))
      4 



ipdb>  c


> /var/folders/x0/r_2wlyls39s3_q5g99w33dn80000gn/T/ipykernel_22918/1017695108.py(45)forward()
     43         **kwargs
     44     ):
1--> 45         return_dict = return_dict if return_dict is not None else self.config.use_return_dict
     46 
     47         if self.use_encoder_parallel:



ipdb>  c


> /var/folders/x0/r_2wlyls39s3_q5g99w33dn80000gn/T/ipykernel_22918/1768784971.py(119)forward()
    117         **kwargs
    118     ):
2-> 119         data_o = self.encode(data_input_ids, data_attention_mask)
    120 
    121         if data_type is not None and data_type == "meta":



...

> /var/folders/x0/r_2wlyls39s3_q5g99w33dn80000gn/T/ipykernel_22918/1017695108.py(75)forward()
     73                                           neg2data_data2ptr, neg2data_idx, plbl2data_data2ptr, plbl2data_idx, **kwargs)
     74 
---> 75             if self.use_calib_loss:
     76                 loss += self.calibration_loss(data_o.fused_rep, data_o.rep, lbl2data_o.rep, lbl2data_data2ptr, lbl2data_idx,
     77                                               plbl2data_data2ptr,plbl2data_idx)



ipdb>  


> /var/folders/x0/r_2wlyls39s3_q5g99w33dn80000gn/T/ipykernel_22918/1017695108.py(81)forward()
     79                                               plbl2data_data2ptr,plbl2data_idx)
     80 
---> 81         if not return_dict:
     82             o = (data_o.logits,data_o.rep,data_o.fused_rep,lbl2data_o.logits,lbl2data_o.rep,lbl2data_o.fused_rep)
     83             return ((loss,) + o) if loss is not None else o



ipdb>  c


    [... skipped 1 hidden frame]

    [... skipped 1 hidden frame]

    [... skipped 1 hidden frame]

    [... skipped 1 hidden frame]



## `OAK16`

In [25]:
#| export
class OAK016(OAK003):

    def __init__(
        self, 
        config,
        neg2data_aug_meta_prefix:Optional[str]=None,
        **kwargs,
    ):
        super().__init__(config, **kwargs)
        store_attr('neg2data_aug_meta_prefix')
        self.rep_loss_fn = MarginMSEWithNegatives()
        
    def compute_loss(self, data_repr, lbl2data_repr, lbl2data_scores, neg2data_repr, neg2data_scores, **kwargs):
        return self.rep_loss_fn(data_repr, lbl2data_repr, lbl2data_scores, neg2data_repr, neg2data_scores, **kwargs)
        
    def forward(
        self,
        data_input_ids:Optional[torch.Tensor]=None,
        data_attention_mask:Optional[torch.Tensor]=None,
        
        lbl2data_input_ids:Optional[torch.Tensor]=None,
        lbl2data_attention_mask:Optional[torch.Tensor]=None,
        lbl2data_data2ptr:Optional[torch.Tensor]=None,
        lbl2data_idx:Optional[torch.Tensor]=None,
        lbl2data_scores:Optional[torch.Tensor]=None,

        neg2data_input_ids:Optional[torch.Tensor]=None,
        neg2data_attention_mask:Optional[torch.Tensor]=None,
        neg2data_data2ptr:Optional[torch.Tensor]=None,
        neg2data_idx:Optional[torch.Tensor]=None,
        neg2data_scores:Optional[torch.Tensor]=None,
        
        plbl2data_data2ptr:Optional[torch.Tensor]=None,
        plbl2data_idx:Optional[torch.Tensor]=None,
        
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
        **kwargs
    ):  
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict
        
        if self.use_encoder_parallel: 
            encoder = XCDataParallel(module=self.encoder)
        else: encoder = self.encoder
        
        data_meta_kwargs = Parameters.from_feat_meta_aug_prefix('data', self.data_aug_meta_prefix, **kwargs)
        data_o = encoder(data_input_ids=data_input_ids, data_attention_mask=data_attention_mask, 
                         data_aug_meta_prefix=self.data_aug_meta_prefix, **data_meta_kwargs)
        
        loss = None; lbl2data_o = EncoderOutput()
        if (
            lbl2data_input_ids is not None and 
            neg2data_input_ids is not None
        ):
            lbl2data_meta_kwargs = Parameters.from_feat_meta_aug_prefix('lbl2data', self.lbl2data_aug_meta_prefix, **kwargs)
            lbl2data_o = encoder(data_input_ids=lbl2data_input_ids, data_attention_mask=lbl2data_attention_mask, 
                                 data_aug_meta_prefix=self.lbl2data_aug_meta_prefix, **lbl2data_meta_kwargs)

            neg2data_meta_kwargs = Parameters.from_feat_meta_aug_prefix('neg2data', self.neg2data_aug_meta_prefix, **kwargs)
            neg2data_o = encoder(data_input_ids=neg2data_input_ids, data_attention_mask=neg2data_attention_mask, 
                                 data_aug_meta_prefix=self.neg2data_aug_meta_prefix, **neg2data_meta_kwargs)

            loss = self.compute_loss(data_o.fused_rep, lbl2data_o.rep, lbl2data_scores, neg2data_o.rep, 
                                     neg2data_scores, **kwargs)
            
            if self.use_query_loss:
                loss += self.compute_loss(data_o.rep, lbl2data_o.rep, lbl2data_scores, neg2data_o.rep, 
                                         neg2data_scores, **kwargs)
                
            if self.use_calib_loss:
                loss += self.calibration_loss(data_o.fused_rep, data_o.rep, lbl2data_o.rep, lbl2data_data2ptr, lbl2data_idx,
                                              plbl2data_data2ptr,plbl2data_idx)
                loss += self.calibration_loss(data_o.fused_rep, data_o.rep, neg2data_o.rep, neg2data_data2ptr, neg2data_idx,
                                              plbl2data_data2ptr,plbl2data_idx)
                
        if not return_dict:
            o = (data_o.logits,data_o.rep,data_o.fused_rep,lbl2data_o.logits,lbl2data_o.rep,lbl2data_o.fused_rep)
            return ((loss,) + o) if loss is not None else o
        
        
        return XCModelOutput(
            loss=loss,
            
            data_repr=data_o.rep,
            data_fused_repr=data_o.fused_rep,
            
            lbl2data_repr=lbl2data_o.rep,
            lbl2data_fused_repr=lbl2data_o.fused_rep,
        )
            

### Example

In [113]:
model = OAK016.from_pretrained('sentence-transformers/msmarco-distilbert-base-v4', margin=0.3, num_negatives=5, 
                               tau=0.1, apply_softmax=True,
                               
                               data_aug_meta_prefix='cat2data', lbl2data_aug_meta_prefix=None,
                               neg2data_aug_meta_prefix=None,
                               
                               num_metadata=block.train.dset.meta['cat_meta'].n_meta, resize_length=5000,
                               
                               calib_margin=0.3, calib_num_negatives=10, calib_tau=0.1, calib_apply_softmax=False, 
                               calib_loss_weight=0.1, use_calib_loss=False,

                               use_query_loss=True,

                               meta_loss_weight=0.3, 
                               
                               fusion_loss_weight=0.1, use_fusion_loss=True,
                               
                               use_encoder_parallel=False, normalize=True)

model.init_retrieval_head()
model.init_cross_head()
model.init_meta_embeddings()

Some weights of OAK016 were not initialized from the model checkpoint at sentence-transformers/msmarco-distilbert-base-v4 and are newly initialized: ['encoder.cross_head.k.bias', 'encoder.cross_head.k.weight', 'encoder.cross_head.o.bias', 'encoder.cross_head.o.weight', 'encoder.cross_head.q.bias', 'encoder.cross_head.q.weight', 'encoder.cross_head.v.bias', 'encoder.cross_head.v.weight', 'encoder.dr_fused_head.layer_norm.bias', 'encoder.dr_fused_head.layer_norm.weight', 'encoder.dr_fused_head.projector.bias', 'encoder.dr_fused_head.projector.weight', 'encoder.dr_fused_head.transform.bias', 'encoder.dr_fused_head.transform.weight', 'encoder.dr_head.layer_norm.bias', 'encoder.dr_head.layer_norm.weight', 'encoder.dr_head.projector.bias', 'encoder.dr_head.projector.weight', 'encoder.dr_head.transform.bias', 'encoder.dr_head.transform.weight', 'encoder.meta_embeddings.weight', 'encoder.meta_head.layer_norm.bias', 'encoder.meta_head.layer_norm.weight', 'encoder.meta_head.projector.bias', 'enc

In [116]:
model.encoder.set_pretrained_meta_embeddings(torch.zeros(656086, 768))
model.encoder.freeze_pretrained_meta_embeddings()

In [ ]:
model = model.to('cuda')

In [119]:
b = prepare_batch(model, batch, m_args=[
    'pcat2data_idx', 'pcat2data_data2ptr', 'cat2data_idx', 'cat2data_input_ids', 'cat2data_attention_mask', 
    'cat2data_data2ptr',
])

In [123]:
o = model(**b.to(model.device))

In [124]:
o.loss

tensor(0.0051, grad_fn=<AddBackward0>)

In [125]:
def func():
    import pdb; pdb.set_trace()
    o = model(**b.to(model.device))
    

In [126]:
func()

> /var/folders/x0/r_2wlyls39s3_q5g99w33dn80000gn/T/ipykernel_22918/3721260802.py(2)func()
      1 def func():
----> 2     import pdb; pdb.set_trace()
      3     o = model(**b.to(model.device))
      4 



ipdb>  b


Num Type         Disp Enb   Where
1   breakpoint   keep no    at /var/folders/x0/r_2wlyls39s3_q5g99w33dn80000gn/T/ipykernel_22918/1017695108.py:45
	breakpoint already hit 3 times
2   breakpoint   keep no    at /var/folders/x0/r_2wlyls39s3_q5g99w33dn80000gn/T/ipykernel_22918/1768784971.py:119
	breakpoint already hit 8 times
3   breakpoint   keep no    at /var/folders/x0/r_2wlyls39s3_q5g99w33dn80000gn/T/ipykernel_22918/2221889608.py:29
	breakpoint already hit 3 times


ipdb>  enable 1


Enabled breakpoint 1 at /var/folders/x0/r_2wlyls39s3_q5g99w33dn80000gn/T/ipykernel_22918/1017695108.py:45


...

tensor(0.0051, grad_fn=<AddBackward0>)


ipdb>  c


    [... skipped 1 hidden frame]

    [... skipped 1 hidden frame]

    [... skipped 1 hidden frame]

    [... skipped 1 hidden frame]

    [... skipped 1 hidden frame]

    [... skipped 1 hidden frame]

    [... skipped 1 hidden frame]

    [... skipped 1 hidden frame]

    [... skipped 1 hidden frame]



IndexError: index 34 is out of bounds for dimension 0 with size 34

    [... skipped 1 hidden frame]

    [... skipped 1 hidden frame]

